# Reverse-Engineering CPS Selective Enrollment Admissions: An MLE Approach

## Recovering Hidden Population Parameters from Truncated Distributions via Maximum Likelihood Estimation, and Constructing a Monte-Carlo Physics Simulation Using These Parameters

**Author:** Andrew Hood  
**Date:** November 2025  
**Data Source:** CPS Official 2024-2025 Cutoffs (released 3/14/2025)

---

### Abstract

Chicago Public Schools (CPS) operates 11 Selective Enrollment High Schools (SEHS) where admission is determined by a composite score (grades + HSAT exam). CPS publishes cutoff scores and average scores of *admitted* students, but crucially, they do not publish the distribution of *all applicants*. Since we only observe students above a threshold, we're dealing with **truncated distributions**. This notebook develops a Maximum Likelihood Estimation (MLE) framework to recover the hidden population parameters $(\mu, \sigma)$ from the published truncated statistics, then uses these recovered parameters to inform a physics-based Monte Carlo simulation of the full admissions process.

You can view this notebook online here:

https://hub.2i2c.mybinder.org/user/homo-morphism-cps_match-j8vb0bcx/notebooks/sehs_analysis_notebook.ipynb

## 1. Background: The CPS Selective Enrollment System

### 1.1 Institutional Context

Chicago Public Schools (CPS) operates 11 Selective Enrollment High Schools (SEHS), the most academically competitive public high schools in the city. These schools admit students based on a **composite score** ranging from 0 to 900 points:

$$\text{Composite Score} = \text{Grades Component (0-450)} + \text{HSAT Score (0-450)}$$

The Grades Component derives from 7th grade coursework in core subjects (Math, English, Science, Social Studies). The HSAT (High School Admissions Test) is a standardized exam administered by CPS each fall.

### 1.2 The Tier System: Designing for Socioeconomic Diversity

CPS faces a fundamental tension: selective schools should admit the highest-achieving students, but without intervention, this would disproportionately favor students from affluent neighborhoods with better-resourced elementary schools. The **tier system** addresses this by segmenting the city into four socioeconomic tiers based on census tract characteristics:

| Tier | Socioeconomic Status | Typical Characteristics |
|------|---------------------|------------------------|
| Tier 1 | Lowest | High poverty rate, low median income, low homeownership |
| Tier 2 | Below average | Moderate poverty, below-average income |
| Tier 3 | Above average | Low poverty, above-average income |
| Tier 4 | Highest | Very low poverty, high income, high educational attainment |

Each census tract is assigned to exactly one tier based on a composite index. Students inherit the tier of their home address, not their elementary school.

### 1.3 Seat Allocation: The 30/70 Split

Each school allocates seats in two phases:

**Phase 1: Rank-Based (30% of seats)**
- The top 30% of seats go to the highest-scoring applicants *citywide*, regardless of tier.
- This rewards absolute academic excellence.
- In practice, these seats are dominated by Tier 3 and Tier 4 students.

**Phase 2: Tier-Based (70% of seats)**
- The remaining 70% is divided equally among the four tiers (17.5% each).
- Within each tier, seats go to the highest-scoring applicants *from that tier*.
- This ensures each socioeconomic group has guaranteed representation.

### 1.4 The Serial Dictatorship Matching Mechanism

CPS uses a **serial dictatorship** (also called **deferred acceptance** in mechanism design) to match students to schools:

1. **Application phase**: Each student submits a ranked list of up to 6 SEHS schools.
2. **Sorting phase**: Students are sorted by composite score (ties broken by lottery).
3. **Matching phase**: Students are processed in score order. Each student is assigned to their highest-ranked school with available seats in their allocation category (Rank or Tier).
4. **Cutoff determination**: The cutoff for each school-tier combination is the score of the last admitted student.

This mechanism is **strategy-proof**: students should truthfully rank schools by preference, not by perceived admission chances. It is also **Pareto-efficient**: no reallocation can make a student better off without making another worse off.

### 1.5 The Data Availability Problem

CPS publishes:
- **Cutoff scores**: Minimum score for admission by tier at each school
- **Average scores**: Mean score of *admitted* students by tier
- **Maximum scores**: Highest score among admitted students by tier

CPS does **not** publish:
- The full distribution of applicant scores (rejected + admitted)
- The number of applicants by tier at each school
- Individual-level application or outcome data

This creates a **truncation problem**: we observe only the right tail of the score distribution (students at or above the cutoff). The published "average" is not the population mean; it is the mean of a truncated distribution.

## 2. The Truncation Problem: Mathematical Framework

### 2.1 Notation and Setup

Let $X$ denote the random variable representing applicant scores within a given tier. I model this as:

$$X \sim \mathcal{N}(\mu, \sigma^2)$$

**Notation:**
- $\mu$ = population mean (the true average score of *all* applicants in this tier, not just those admitted)
- $\sigma$ = population standard deviation (the spread of scores among all applicants)
- $c$ = cutoff score (the minimum score for admission)
- $M$ = maximum possible score (900 in the CPS system)

**Justification for the normality assumption**: Composite scores are the sum of two components, each aggregating multiple sub-scores (grades across subjects, exam scores across sections). By the Central Limit Theorem, such sums tend toward normality for large populations. This assumption is standard in educational measurement. However, I later relax this in the simulation by using skewed normal distributions to capture empirical deviations from symmetry.

### 2.2 The Truncated Distribution

CPS admits students with $X \geq c$, where $c$ is the cutoff. The *observed* data comes from the **truncated distribution**:

$$X \mid X \geq c$$

The critical insight is that the observed mean is systematically biased upward:

$$\bar{X}_{\text{observed}} = \mathbb{E}[X \mid X \geq c] > \mu$$

This bias is called **selection bias** or **truncation bias**. It arises because we only see high scorers; the full population includes many students who scored below the cutoff and were rejected.

### 2.3 The Truncated Normal Mean Formula

For a normal distribution truncated to the interval $[c, M]$, the conditional expectation is:

$$\mathbb{E}[X \mid c \leq X \leq M] = \mu + \sigma \cdot \frac{\phi(\alpha) - \phi(\beta)}{\Phi(\beta) - \Phi(\alpha)}$$

where:
- $\alpha = \frac{c - \mu}{\sigma}$ is the standardized lower bound (cutoff in $z$-score units)
- $\beta = \frac{M - \mu}{\sigma}$ is the standardized upper bound
- $\phi(\cdot)$ is the standard normal PDF: $\phi(z) = \frac{1}{\sqrt{2\pi}} e^{-z^2/2}$
- $\Phi(\cdot)$ is the standard normal CDF: $\Phi(z) = \int_{-\infty}^{z} \phi(t) \, dt$

The ratio $\lambda(\alpha) = \frac{\phi(\alpha)}{1 - \Phi(\alpha)}$ is the **inverse Mills ratio** (or **hazard rate** of the standard normal). It quantifies how much truncation inflates the observed mean. When $\alpha$ is large (cutoff far above the mean), $\lambda(\alpha)$ is large, and selection bias is severe.

### 2.4 The Identification Problem

Given only:
- The truncated mean $\bar{X}_{\text{obs}}$ (published by CPS as "average score")
- The cutoff $c$ (published by CPS)

I have **one equation in two unknowns** $(\mu, \sigma)$. The locus of solutions forms a curve (often called a "banana" due to its shape in parameter space). Many different $(\mu, \sigma)$ pairs produce the same truncated mean.

**Solution: Add a second constraint.** I use the **acceptance rate**:

$$P(X \geq c) = 1 - \Phi\left(\frac{c - \mu}{\sigma}\right) = \frac{\text{seats}}{\text{applicants}} \equiv r$$

With two independent constraints (truncated mean and acceptance rate), I can uniquely identify both $\mu$ and $\sigma$.

**Caveat**: CPS does not publish tier-specific applicant counts. I estimate the acceptance rate $r$ using total seat counts and city-wide applicant estimates. This introduces some uncertainty, but sensitivity analysis shows the MLE results are robust to reasonable variations in $r$.

In [ ]:
# ==============================================================================
# LIBRARY IMPORTS AND CONFIGURATION
# ==============================================================================
#
# This cell loads all required libraries for the analysis:
#   - numpy: Array operations and random number generation
#   - pandas: DataFrame manipulation for tabular data
#   - matplotlib/seaborn: Visualization
#   - scipy.stats: Statistical distributions (norm, skewnorm) and optimization
#   - dataclasses: Structured containers for tier data and results
#
# I suppress optimization warnings because L-BFGS-B may not converge perfectly
# for edge cases (e.g., schools with very low cutoffs), but the results are
# still usable.
# ==============================================================================

import numpy as np                          # Numerical array operations
import pandas as pd                         # DataFrame manipulation
import matplotlib.pyplot as plt             # Core plotting library
import seaborn as sns                       # Statistical visualization
from scipy import stats, optimize           # Distributions and optimization
from scipy.stats import skewnorm            # Skewed normal distribution for simulation
from dataclasses import dataclass           # Structured data containers
from typing import Optional, Dict, List, Tuple, Callable
import warnings

# Suppress convergence warnings from optimizer
# These occur for schools with extreme parameters (e.g., very tight T4 at elite schools)
# and do not affect the validity of results
warnings.filterwarnings('ignore')

# Configure plot aesthetics for publication-quality figures
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.1)

# Set global random seed for reproducibility
# All stochastic operations (simulation, Monte Carlo validation) use this seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("Libraries loaded successfully.")

## 3. MLE Framework Implementation

### 3.1 Core Mathematical Functions

I now implement the truncated normal distribution formulas from Section 2. The two key functions are:

1. **`truncated_mean(mu, sigma, lower, upper)`**: Computes $\mathbb{E}[X \mid c \leq X \leq M]$
2. **`acceptance_prob(mu, sigma, cutoff)`**: Computes $P(X \geq c) = 1 - \Phi\left(\frac{c-\mu}{\sigma}\right)$

**Variable-to-LaTeX correspondence:**

| Code Variable | LaTeX Symbol | Meaning |
|---------------|--------------|---------|
| `mu` | $\mu$ | Population mean (hidden, to be estimated) |
| `sigma` | $\sigma$ | Population standard deviation (hidden) |
| `lower` | $c$ | Cutoff score (lower truncation point) |
| `upper` | $M$ | Maximum score (900; upper truncation point) |
| `alpha` | $\alpha$ | Standardized lower bound: $(c - \mu)/\sigma$ |
| `beta` | $\beta$ | Standardized upper bound: $(M - \mu)/\sigma$ |
| `phi` | $\phi$ | Standard normal PDF |
| `Phi` | $\Phi$ | Standard normal CDF |

In [ ]:
# ==============================================================================
# GLOBAL CONSTANTS
# ==============================================================================
# M: Maximum possible composite score in the CPS system.
#    This acts as the implicit upper bound for truncation.
#    In LaTeX: M = 900
# ==============================================================================
MAX_SCORE = 900  # M in LaTeX notation


def truncated_mean(mu: float, sigma: float, lower: float, upper: float = MAX_SCORE) -> float:
    """
    Compute the expected value of a normal distribution truncated to [lower, upper].
    
    Implements the formula:
        E[X | c <= X <= M] = mu + sigma * (phi(alpha) - phi(beta)) / (Phi(beta) - Phi(alpha))
    
    where:
        alpha = (c - mu) / sigma    (standardized lower bound)
        beta  = (M - mu) / sigma    (standardized upper bound)
        phi   = standard normal PDF
        Phi   = standard normal CDF
    
    Parameters
    ----------
    mu : float
        Population mean of the untruncated distribution (the hidden parameter we estimate).
        In LaTeX: mu
    sigma : float
        Population standard deviation (the hidden spread parameter).
        In LaTeX: sigma
    lower : float
        Lower truncation point (the cutoff score c).
        In LaTeX: c
    upper : float
        Upper truncation point (default: MAX_SCORE = 900).
        In LaTeX: M
    
    Returns
    -------
    float
        The conditional expectation E[X | c <= X <= M].
    
    Notes
    -----
    When the probability mass in [lower, upper] is negligible (< 1e-10),
    the function returns NaN to signal a degenerate case.
    """
    # Compute standardized bounds
    # alpha = (c - mu) / sigma: how many standard deviations the cutoff is above the mean
    # beta = (M - mu) / sigma: how many standard deviations the max score is above the mean
    alpha = (lower - mu) / sigma
    beta = (upper - mu) / sigma
    
    # Get standard normal PDF (phi) and CDF (Phi) from scipy
    phi = stats.norm.pdf  # phi(z) = (1/sqrt(2*pi)) * exp(-z^2 / 2)
    Phi = stats.norm.cdf  # Phi(z) = integral_{-inf}^{z} phi(t) dt
    
    # Compute probability mass in the truncation region
    # This is P(c <= X <= M) = Phi(beta) - Phi(alpha)
    prob_mass = Phi(beta) - Phi(alpha)
    
    # Handle degenerate case: if almost no probability mass in region, return NaN
    # This can happen if cutoff is extremely high relative to the mean
    if prob_mass < 1e-10:
        return np.nan
    
    # Apply the truncated mean formula
    # The numerator (phi(alpha) - phi(beta)) / prob_mass is the "correction factor"
    # that quantifies the selection bias from observing only the truncated region
    truncated_expectation = mu + sigma * (phi(alpha) - phi(beta)) / prob_mass
    
    return truncated_expectation


def acceptance_prob(mu: float, sigma: float, cutoff: float) -> float:
    """
    Compute the probability that a randomly drawn score exceeds the cutoff.
    
    Implements:
        P(X >= c) = 1 - Phi((c - mu) / sigma)
    
    This represents the fraction of applicants who score at or above the cutoff,
    i.e., the acceptance rate if all applicants above the cutoff were admitted.
    
    Parameters
    ----------
    mu : float
        Population mean. In LaTeX: mu
    sigma : float
        Population standard deviation. In LaTeX: sigma
    cutoff : float
        Admission cutoff score. In LaTeX: c
    
    Returns
    -------
    float
        P(X >= c), the probability of exceeding the cutoff.
    
    Notes
    -----
    This is the second constraint used to identify (mu, sigma).
    The target acceptance rate r = seats / applicants.
    """
    # P(X >= c) = 1 - P(X < c) = 1 - Phi((c - mu) / sigma)
    # Using scipy's norm.cdf with loc=mu, scale=sigma
    return 1.0 - stats.norm.cdf(cutoff, loc=mu, scale=sigma)

### 3.2 Demonstrating Selection Bias

Before fitting the full model, I demonstrate the magnitude of selection bias with a concrete example. Consider a hypothetical tier where:
- True population mean $\mu = 700$ points
- True population standard deviation $\sigma = 80$ points
- Cutoff score $c = 800$ points

The cutoff is 1.25 standard deviations above the mean, so only about 10% of applicants are admitted. For those who *are* admitted, what is their average score?

The truncated mean will be **substantially higher** than 700 because we only observe the right tail of the distribution.

In [ ]:
# ==============================================================================
# DEMONSTRATION: SELECTION BIAS IN TRUNCATED DISTRIBUTIONS
# ==============================================================================
# This example shows how truncation inflates the observed mean.
#
# Variables (LaTeX correspondence):
#   mu_true    -> mu     : true population mean (700)
#   sigma_true -> sigma  : true population std dev (80)
#   cutoff     -> c      : admission cutoff (800)
#   trunc_mean -> E[X|X>=c] : expected value given admission
#   accept_rate -> P(X>=c)  : probability of admission
#   selection_bias -> E[X|X>=c] - mu : upward bias from truncation
# ==============================================================================

# Define hypothetical population parameters
# These represent the TRUE (hidden) distribution of ALL applicants
mu_true = 700       # mu: True population mean
sigma_true = 80     # sigma: True population standard deviation
cutoff = 800        # c: Admission cutoff (only X >= 800 are admitted)

# Compute the truncated mean using our formula
# This is what CPS would publish as the "average admitted score"
trunc_mean = truncated_mean(mu_true, sigma_true, cutoff)

# Selection bias: the difference between observed and true mean
# This quantifies how much truncation inflates the published average
selection_bias = trunc_mean - mu_true

# Compute acceptance probability: P(X >= c)
# This is the fraction of applicants who get admitted
accept_rate = acceptance_prob(mu_true, sigma_true, cutoff)

# Display results
print("SELECTION BIAS DEMONSTRATION")
print("=" * 60)
print(f"True population mean (mu):           {mu_true}")
print(f"True population std dev (sigma):     {sigma_true}")
print(f"Cutoff score (c):                    {cutoff}")
print(f"Z-score of cutoff: (c - mu)/sigma =  {(cutoff - mu_true)/sigma_true:.2f}")
print()
print(f"Truncated mean E[X | X >= c]:        {trunc_mean:.1f}")
print(f"Selection bias (E[X|X>=c] - mu):     +{selection_bias:.1f} points")
print(f"Acceptance rate P(X >= c):           {accept_rate:.1%}")
print()
print("INTERPRETATION:")
print(f"  If CPS published an 'average score' of {trunc_mean:.0f}, a naive reader")
print(f"  might conclude the typical applicant scores around {trunc_mean:.0f}.")
print(f"  In fact, the true population mean is only {mu_true}.")
print(f"  The {selection_bias:.0f}-point gap is pure selection bias.")

### 3.3 Data Structures

I define two dataclasses to organize the analysis:

**TierData**: Holds the *observed* (public) data for a single tier at a school.
- `cutoff`: The published minimum score $c$ for admission
- `observed_mean`: The published average score $\bar{X}_{\text{obs}}$ of admitted students
- `seats` and `applicants`: Used to compute the target acceptance rate $r$

**FittedTier**: Holds the *estimated* (recovered) parameters after MLE.
- `mu` and `sigma`: The MLE estimates $\hat{\mu}$ and $\hat{\sigma}$
- `selection_bias`: The difference $\bar{X}_{\text{obs}} - \hat{\mu}$

**Variable-to-LaTeX correspondence:**

| Code Variable | LaTeX Symbol | Meaning |
|---------------|--------------|---------|
| `cutoff` | $c$ | Admission cutoff (observed) |
| `observed_mean` | $\bar{X}_{\text{obs}}$ | Published average of admitted students |
| `seats` | - | Number of seats at the school for this tier |
| `applicants` | - | Estimated applicants for this tier |
| `target_accept_rate` | $r$ | seats / applicants |
| `mu` | $\hat{\mu}$ | MLE estimate of population mean |
| `sigma` | $\hat{\sigma}$ | MLE estimate of population std dev |
| `selection_bias` | $\bar{X}_{\text{obs}} - \hat{\mu}$ | Truncation-induced bias |

In [ ]:
# ==============================================================================
# DATA STRUCTURES FOR MLE ANALYSIS
# ==============================================================================
# These dataclasses organize the input data and output results.
# Using dataclasses provides type safety and clean attribute access.
# ==============================================================================

@dataclass
class TierData:
    """
    Container for OBSERVED (public) data about a single tier at a school.
    
    This represents information CPS publishes. The goal of MLE is to recover
    the hidden parameters (mu, sigma) that generated this observed data.
    
    Attributes
    ----------
    name : str
        Tier identifier, e.g., "Tier 1", "Tier 4"
    
    cutoff : float
        Minimum score for admission (c in LaTeX).
        This is the score of the last admitted student.
    
    observed_mean : float or None
        Mean score of ADMITTED students (X_bar_obs in LaTeX).
        IMPORTANT: This is NOT the population mean. It is the truncated mean,
        which is systematically higher than mu due to selection.
    
    seats : float
        Number of seats allocated to this tier at this school.
        CPS allocates 17.5% of seats to each tier (70% total / 4 tiers).
    
    applicants : float
        Estimated number of applicants from this tier.
        CPS does not publish this; I estimate from total applicants / 4.
    """
    name: str
    cutoff: float
    observed_mean: Optional[float]
    seats: float
    applicants: float
    
    @property
    def target_accept_rate(self) -> float:
        """
        Compute target acceptance rate r = seats / applicants.
        
        This is the second constraint for MLE. If we know r and c, combined
        with the truncated mean constraint, we can uniquely identify (mu, sigma).
        
        Returns
        -------
        float
            The implied acceptance rate (fraction of applicants admitted).
        """
        return self.seats / self.applicants


@dataclass  
class FittedTier:
    """
    Container for MLE ESTIMATION RESULTS for a single tier.
    
    After optimization, this holds the recovered hidden parameters.
    
    Attributes
    ----------
    name : str
        Tier identifier (copied from input TierData)
    
    mu : float
        MLE estimate of population mean (mu_hat in LaTeX).
        This is the TRUE average score of ALL applicants in this tier,
        not just those who were admitted.
    
    sigma : float
        MLE estimate of population standard deviation (sigma_hat).
        This measures the spread of scores among ALL applicants.
    
    cutoff : float
        Cutoff score (copied from input; c in LaTeX)
    
    fitted_mean : float
        Truncated mean computed from fitted (mu_hat, sigma_hat).
        Should match observed_mean if the fit is good.
    
    fitted_accept_rate : float
        Acceptance probability computed from fitted parameters.
        Should match target_accept_rate if the fit is good.
    
    selection_bias : float
        Difference between observed_mean and mu_hat.
        Quantifies how much truncation inflates the published average.
        Larger bias indicates more severe truncation (cutoff far above mean).
    """
    name: str
    mu: float
    sigma: float
    cutoff: float
    fitted_mean: float
    fitted_accept_rate: float
    selection_bias: float

### 3.4 The MLE Fitting Function

I recover $(\mu, \sigma)$ by minimizing a weighted loss function that combines two constraints:

$$\mathcal{L}(\mu, \sigma) = \underbrace{\left(\mathbb{E}[X|X \geq c; \mu, \sigma] - \bar{X}_{\text{obs}}\right)^2}_{\text{Constraint 1: Match truncated mean}} + \lambda \underbrace{\left(P(X \geq c; \mu, \sigma) - r\right)^2}_{\text{Constraint 2: Match acceptance rate}}$$

where:
- $\bar{X}_{\text{obs}}$ is the observed (truncated) mean from CPS data
- $r = \text{seats} / \text{applicants}$ is the target acceptance rate
- $\lambda = 100$ is a weighting parameter

**Why $\lambda = 100$?** The two constraint terms have different scales:
- Mean errors are typically $O(1)$ to $O(10)$ points, so squared errors are $O(1)$ to $O(100)$
- Acceptance rate errors are typically $O(0.01)$ to $O(0.1)$, so squared errors are $O(0.0001)$ to $O(0.01)$

Without weighting, the optimizer would prioritize the mean constraint and ignore the acceptance rate. Setting $\lambda = 100$ brings both terms to comparable scales, ensuring both constraints influence the solution.

**Optimization details:**
- I use L-BFGS-B, a quasi-Newton method that supports box constraints
- $\mu$ is constrained to $[100, 890]$ (must be below the max score)
- $\sigma$ is constrained to $[5, 200]$ (prevents degenerate solutions)
- Initial guesses are tier-specific: lower $\mu$ for Tier 1, higher for Tier 4

In [ ]:
# ==============================================================================
# MAXIMUM LIKELIHOOD ESTIMATION FUNCTION
# ==============================================================================
# This function recovers the hidden parameters (mu, sigma) by minimizing a
# weighted sum of squared constraint violations.
#
# Variables (LaTeX correspondence):
#   mu        -> mu          : population mean (parameter to estimate)
#   sigma     -> sigma       : population std dev (parameter to estimate)
#   theor_mean -> E[X|X>=c]  : theoretical truncated mean given (mu, sigma)
#   theor_accept -> P(X>=c)  : theoretical acceptance prob given (mu, sigma)
#   mean_error -> (E[X|X>=c] - X_bar_obs)^2 : squared error for mean constraint
#   accept_error -> (P(X>=c) - r)^2        : squared error for acceptance constraint
#   accept_weight -> lambda  : weighting parameter (100)
# ==============================================================================

def fit_tier_mle(tier: TierData, 
                 use_accept_rate: bool = True,
                 accept_weight: float = 100.0) -> FittedTier:
    """
    Recover hidden population parameters (mu, sigma) via constrained optimization.
    
    The objective function is:
        L(mu, sigma) = (E[X|X>=c] - X_obs)^2 + lambda * (P(X>=c) - r)^2
    
    where lambda = accept_weight balances the two constraints.
    
    Parameters
    ----------
    tier : TierData
        Observed data for this tier (cutoff, mean, seats, applicants)
    
    use_accept_rate : bool, default=True
        Whether to include the acceptance rate constraint.
        If False, only the mean constraint is used (underidentified).
    
    accept_weight : float, default=100.0
        Weight lambda on the acceptance rate constraint.
        Chosen to balance the scales of mean errors (~O(1-10)) and
        acceptance rate errors (~O(0.01-0.1)).
    
    Returns
    -------
    FittedTier
        MLE estimates (mu_hat, sigma_hat) and derived quantities.
    """
    
    def loss(params: np.ndarray) -> float:
        """
        Objective function to minimize.
        
        L(mu, sigma) = mean_error + lambda * accept_error
        
        Parameters
        ----------
        params : ndarray of shape (2,)
            params[0] = mu (population mean)
            params[1] = sigma (population std dev)
        
        Returns
        -------
        float
            Total loss value (sum of squared constraint violations)
        """
        mu, sigma = params
        
        # Reject invalid parameter values with a large penalty
        # sigma must be positive; mu must be in a reasonable range
        if sigma <= 0 or mu <= 0 or mu >= MAX_SCORE:
            return 1e10
        
        total_loss = 0.0
        
        # ---------------------------------------------------------------------
        # CONSTRAINT 1: Truncated mean must match observed mean
        # ---------------------------------------------------------------------
        # The truncated mean E[X|X>=c] depends on (mu, sigma) and the cutoff c.
        # We want to find (mu, sigma) such that this matches the published average.
        if tier.observed_mean is not None:
            # Compute E[X | X >= c] using the formula from Section 2.3
            theor_mean = truncated_mean(mu, sigma, tier.cutoff)
            
            # Handle numerical issues (e.g., if cutoff is way above mu)
            if np.isnan(theor_mean):
                return 1e10
            
            # Squared error: (E[X|X>=c] - X_bar_obs)^2
            mean_error = (theor_mean - tier.observed_mean) ** 2
            total_loss += mean_error
        
        # ---------------------------------------------------------------------
        # CONSTRAINT 2: Acceptance rate must match target
        # ---------------------------------------------------------------------
        # P(X >= c) = 1 - Phi((c - mu)/sigma) should equal seats/applicants
        if use_accept_rate:
            # Compute P(X >= c)
            theor_accept = acceptance_prob(mu, sigma, tier.cutoff)
            
            # Squared error: (P(X>=c) - r)^2
            accept_error = (theor_accept - tier.target_accept_rate) ** 2
            
            # Apply weighting to balance scales
            # lambda = 100 makes accept_error comparable to mean_error
            total_loss += accept_weight * accept_error
        
        return total_loss
    
    # -------------------------------------------------------------------------
    # INITIAL GUESSES (tier-specific to improve convergence)
    # -------------------------------------------------------------------------
    # Tier 1 (disadvantaged): Expect lower mean, wider spread
    # Tier 4 (affluent): Expect higher mean, tighter spread
    # These are rough guesses; the optimizer will refine them.
    if "1" in tier.name:
        start_mu, start_sigma = 600, 100
    elif "4" in tier.name:
        start_mu, start_sigma = 820, 50
    else:
        start_mu, start_sigma = 700, 70
    
    # -------------------------------------------------------------------------
    # RUN OPTIMIZATION
    # -------------------------------------------------------------------------
    # L-BFGS-B is a quasi-Newton method that supports box constraints.
    # It approximates the Hessian using gradient information.
    result = optimize.minimize(
        loss,
        x0=[start_mu, start_sigma],     # Starting point
        method='L-BFGS-B',              # Optimizer choice
        bounds=[(100, 890), (5, 200)]   # Box constraints on (mu, sigma)
    )
    
    # Extract the MLE estimates
    mu_hat, sigma_hat = result.x
    
    # -------------------------------------------------------------------------
    # COMPUTE DERIVED QUANTITIES FOR VALIDATION
    # -------------------------------------------------------------------------
    # These allow us to verify the fit quality
    fitted_mean = truncated_mean(mu_hat, sigma_hat, tier.cutoff)
    fitted_accept = acceptance_prob(mu_hat, sigma_hat, tier.cutoff)
    
    # Selection bias: how much higher the observed mean is than the true mean
    # This is the key quantity we are recovering
    if tier.observed_mean is not None:
        selection_bias = tier.observed_mean - mu_hat
    else:
        selection_bias = fitted_mean - mu_hat
    
    return FittedTier(
        name=tier.name,
        mu=mu_hat,
        sigma=sigma_hat,
        cutoff=tier.cutoff,
        fitted_mean=fitted_mean,
        fitted_accept_rate=fitted_accept,
        selection_bias=selection_bias
    )

## 4. MLE Analysis: All 11 SEHS Schools

### 4.1 Data from CPS 2024-2025 Report

The following data comes from the official CPS document *"Initial Offer Point Totals for Selective Enrollment High Schools 2025-2026"*, released March 14, 2025. For each school and tier, I extracted:
- **Cutoff** ($c$): Minimum score for admission (the "Min Point Total" column)
- **Average** ($\bar{X}_{\text{obs}}$): Mean score of admitted students (the "Average Point Total" column)

**Assumptions about applicant counts:**

CPS does not publish tier-specific applicant counts. I estimate these as follows:
1. **Total applicants**: I use historical enrollment data and reported application volumes. For highly competitive schools (Payton, Lane Tech), applicant counts are much higher relative to seat counts.
2. **Per-tier distribution**: I assume applicants are uniformly distributed across tiers (25% each). This is a simplification; in reality, Tier 4 may be over-represented at elite North Side schools.

These estimates affect the acceptance rate constraint but not the truncated mean constraint. Sensitivity analysis shows the MLE results are robust to $\pm 20\%$ variations in applicant estimates.

In [ ]:
# ==============================================================================
# CPS 2024-2025 OFFICIAL DATA
# ==============================================================================
# Source: "Initial Offer Point Totals for Selective Enrollment High Schools 2025-2026"
#         Published by Chicago Public Schools, released March 14, 2025
#
# Data structure for each school:
#   seats           : Total freshmen seats at the school
#   total_applicants: Estimated total applicants (CPS does not publish this directly)
#   tiers           : Dict mapping tier number (1-4) to {cutoff, avg}
#       cutoff = "Min Point Total" from CPS report (admission threshold c)
#       avg    = "Average Point Total" from CPS report (truncated mean X_bar_obs)
#
# Schools are grouped by selectivity:
#   ELITE (North Side / Loop): Payton, Northside, Whitney Young, Jones, Lane Tech
#   REGIONAL (South Side / West Side): Hancock, Lindblom, Brooks, Westinghouse, King, South Shore
# ==============================================================================

SCHOOL_DATA = {
    # =========================================================================
    # ELITE SCHOOLS (North Side / Loop)
    # These schools have the highest cutoffs and most competitive T4 pools.
    # T4 applicants at these schools often score 850-900.
    # =========================================================================
    
    'Lane Tech': {
        # Largest SEHS (1200 seats). Strong demand from North Side T3/T4.
        # Lane has relatively balanced tier demand due to broad geographic draw.
        'seats': 1200,
        'total_applicants': 13000,  # High volume due to prestige + large class
        'tiers': {
            1: {'cutoff': 712, 'avg': 758.2},   # T1 cutoff ~800s at elite schools
            2: {'cutoff': 780, 'avg': 814.5},
            3: {'cutoff': 817.5, 'avg': 843.1},
            4: {'cutoff': 859, 'avg': 867.1},   # T4 very competitive
        }
    },
    
    'Walter Payton': {
        # Most competitive SEHS. T4 cutoff often approaches 900.
        # Small class (350 seats) intensifies competition.
        'seats': 350,
        'total_applicants': 9000,  # Very high applications despite small size
        'tiers': {
            1: {'cutoff': 796, 'avg': 841.3},   # Even T1 requires ~800
            2: {'cutoff': 864, 'avg': 885.5},
            3: {'cutoff': 873, 'avg': 887.5},
            4: {'cutoff': 898, 'avg': 899.7},   # Near-perfect scores required
        }
    },
    
    'Northside': {
        # Second most competitive. Very tight T4 distribution.
        # Strong preference from North Side families.
        'seats': 300,
        'total_applicants': 7000,
        'tiers': {
            1: {'cutoff': 706.5, 'avg': 768.5},
            2: {'cutoff': 841, 'avg': 866.6},
            3: {'cutoff': 861, 'avg': 879.9},
            4: {'cutoff': 893, 'avg': 894.3},   # Extremely tight; avg only 1 point above cutoff
        }
    },
    
    'Whitney Young': {
        # Downtown location draws from all regions.
        # More diverse applicant pool geographically.
        'seats': 350,
        'total_applicants': 8000,
        'tiers': {
            1: {'cutoff': 807, 'avg': 846.0},
            2: {'cutoff': 832, 'avg': 861.2},
            3: {'cutoff': 861, 'avg': 875.2},
            4: {'cutoff': 880, 'avg': 887.4},
        }
    },
    
    'Jones': {
        # Loop location. Pre-engineering and pre-law tracks attract specific applicants.
        # Generally similar to Whitney Young in competitiveness.
        'seats': 375,
        'total_applicants': 7500,
        'tiers': {
            1: {'cutoff': 775, 'avg': 815.7},
            2: {'cutoff': 825, 'avg': 846.0},
            3: {'cutoff': 834, 'avg': 852.7},
            4: {'cutoff': 864, 'avg': 871.1},
        }
    },
    
    # =========================================================================
    # REGIONAL SCHOOLS (South Side / West Side)
    # Lower cutoffs reflect different applicant pools. Many of these schools
    # draw primarily from their local community. Cutoffs can be 100-300 points
    # lower than elite schools.
    # =========================================================================
    
    'Hancock': {
        # Northeast location (Bridgeport area). Draws from South Side T4.
        # Note: T4 cutoff < T3 cutoff (unusual pattern due to applicant pool composition)
        'seats': 250,
        'total_applicants': 4000,
        'tiers': {
            1: {'cutoff': 746, 'avg': 779.3},
            2: {'cutoff': 791, 'avg': 813.2},
            3: {'cutoff': 805, 'avg': 821.7},
            4: {'cutoff': 773, 'avg': 807.9},   # T4 < T3: high-scoring T4 go to elite schools
        }
    },
    
    'Lindblom': {
        # South Side (Englewood). Has Academic Center feeding program.
        # T4 cutoff is notably low, reflecting South Side T4 pool dynamics.
        'seats': 300,
        'total_applicants': 4500,
        'tiers': {
            1: {'cutoff': 691, 'avg': 720.1},
            2: {'cutoff': 707, 'avg': 728.5},
            3: {'cutoff': 725, 'avg': 745.4},
            4: {'cutoff': 600.5, 'avg': 667.4},  # Very low T4 cutoff
        }
    },
    
    'Brooks': {
        # Far South Side (Roseland). Similar dynamics to Lindblom.
        # T4 cutoff again below T3, consistent with elite school siphoning.
        'seats': 350,
        'total_applicants': 4000,
        'tiers': {
            1: {'cutoff': 689.5, 'avg': 729.0},
            2: {'cutoff': 737, 'avg': 764.7},
            3: {'cutoff': 761, 'avg': 782.1},
            4: {'cutoff': 706.5, 'avg': 747.7},
        }
    },
    
    'Westinghouse': {
        # West Side (East Garfield Park). Draws primarily from West Side.
        # T4 cutoff reflects West Side T4 pool (higher than South Side T4).
        'seats': 300,
        'total_applicants': 3500,
        'tiers': {
            1: {'cutoff': 662.5, 'avg': 700.2},
            2: {'cutoff': 699.5, 'avg': 730.7},
            3: {'cutoff': 689.5, 'avg': 726.9},  # T3 < T2 (unusual)
            4: {'cutoff': 635.5, 'avg': 703.6},
        }
    },
    
    'King': {
        # South Side (Bronzeville). Has the lowest cutoffs in the system.
        # Reflects the socioeconomic challenges of the surrounding community.
        'seats': 250,
        'total_applicants': 3000,
        'tiers': {
            1: {'cutoff': 507, 'avg': 567.0},   # Lowest T1 cutoff
            2: {'cutoff': 518, 'avg': 569.0},
            3: {'cutoff': 514.5, 'avg': 577.5},
            4: {'cutoff': 507.5, 'avg': 573.1}, # T4 essentially same as T1
        }
    },
    
    'South Shore': {
        # Far South Side. Second lowest cutoffs after King.
        # Small class size (200 seats).
        'seats': 200,
        'total_applicants': 2500,
        'tiers': {
            1: {'cutoff': 530, 'avg': 587.9},
            2: {'cutoff': 536.5, 'avg': 583.6},  # T2 avg < T1 avg (unusual)
            3: {'cutoff': 525.5, 'avg': 586.6},
            4: {'cutoff': 503.5, 'avg': 568.1},
        }
    },
}

print(f"Loaded data for {len(SCHOOL_DATA)} SEHS schools.")
print(f"Total seats across all schools: {sum(s['seats'] for s in SCHOOL_DATA.values()):,}")

In [ ]:
# ==============================================================================
# RUN MLE FOR ALL SCHOOLS
# ==============================================================================
# This function applies the MLE procedure to all four tiers of a school.
#
# Variables:
#   school_name     : Name of the school (string key into SCHOOL_DATA)
#   school_info     : Dict with seats, total_applicants, tiers
#   tier_seats      : Number of seats allocated to each tier (17.5% of total)
#   tier_applicants : Estimated applicants per tier (total / 4, assuming uniform)
#   tier_data       : TierData object constructed from CPS published numbers
#   results         : Dict mapping tier number (1-4) to FittedTier objects
# ==============================================================================

def run_mle_for_school(school_name: str, school_info: dict) -> Dict[int, FittedTier]:
    """
    Run MLE analysis for all four tiers of a single school.
    
    Parameters
    ----------
    school_name : str
        Name of the school (used for logging only)
    school_info : dict
        Must contain:
        - 'seats': Total seats at the school
        - 'total_applicants': Estimated total applicants
        - 'tiers': Dict mapping tier (1-4) to {'cutoff': c, 'avg': X_bar_obs}
    
    Returns
    -------
    Dict[int, FittedTier]
        Mapping from tier number to fitted MLE results.
    """
    results = {}
    
    # -------------------------------------------------------------------------
    # COMPUTE SEAT AND APPLICANT ALLOCATIONS
    # -------------------------------------------------------------------------
    # CPS allocates 17.5% of seats to each tier (70% total / 4 tiers)
    # The remaining 30% goes to rank-based admission (top scorers citywide)
    tier_seats = school_info['seats'] * 0.175
    
    # Assume applicants are uniformly distributed across tiers
    # This is a simplification; reality is more complex (see Section 4.1)
    tier_applicants = school_info['total_applicants'] / 4
    
    # -------------------------------------------------------------------------
    # FIT MLE FOR EACH TIER
    # -------------------------------------------------------------------------
    for tier_num, tier_info in school_info['tiers'].items():
        # Construct TierData object from CPS published numbers
        # cutoff = c (minimum score for admission)
        # observed_mean = X_bar_obs (average of admitted students)
        tier_data = TierData(
            name=f"Tier {tier_num}",
            cutoff=tier_info['cutoff'],          # c from CPS report
            observed_mean=tier_info['avg'],      # X_bar_obs from CPS report
            seats=tier_seats,                    # 17.5% of total seats
            applicants=tier_applicants           # ~25% of total applicants
        )
        
        # Run the MLE optimization
        # This returns estimates (mu_hat, sigma_hat) for this tier
        results[tier_num] = fit_tier_mle(tier_data)
    
    return results


# ==============================================================================
# EXECUTE MLE FOR ALL 11 SCHOOLS
# ==============================================================================
# Store results in a dictionary: school_name -> {tier: FittedTier}
ALL_MLE_RESULTS = {}

for school_name, school_info in SCHOOL_DATA.items():
    ALL_MLE_RESULTS[school_name] = run_mle_for_school(school_name, school_info)

print("MLE analysis complete for all schools.")
print(f"Total tier-level estimates: {sum(len(r) for r in ALL_MLE_RESULTS.values())}")

### 4.2 Results Summary

The table below shows the recovered hidden parameters for **Tier 4** across all schools, sorted by estimated population mean $\hat{\mu}$. I focus on T4 because it reveals the most striking differences between elite and regional schools.

**Key observations:**

1. **Elite schools have extremely tight T4 distributions.** Northside has $\hat{\sigma} < 5$, meaning virtually all T4 applicants who rank Northside score in a narrow 10-15 point band near 890-900. This reflects both extreme competition and the 900-point ceiling.

2. **Regional schools have much wider T4 spreads.** King and South Shore have $\hat{\sigma} > 60$, indicating heterogeneous T4 applicant pools with scores spanning 200+ points.

3. **The $\hat{\mu}$ gap between elite and regional schools is enormous.** Elite T4 $\hat{\mu}$ ranges from 830-890; regional T4 $\hat{\mu}$ ranges from 590-720. This 200+ point gap reflects the fundamental bifurcation in Chicago's school system.

4. **Some schools hit optimization bounds.** When $\hat{\sigma}$ equals exactly 5 or 200, it suggests the true distribution may deviate from normality (e.g., bimodal or heavily skewed).

In [ ]:
# ==============================================================================
# CREATE SUMMARY TABLE OF MLE RESULTS
# ==============================================================================
# This cell compiles all tier-level MLE results into a pandas DataFrame
# for easier analysis and display.
#
# Variables:
#   summary_rows   : List of dicts, each representing one tier at one school
#   mle_summary    : DataFrame with all tier-level results
#   t4_summary     : DataFrame filtered to Tier 4, sorted by Hidden_mu
#
# Columns in the summary:
#   School         : School name
#   Tier           : Tier number (1-4)
#   Hidden_mu      : MLE estimate of population mean (mu_hat)
#   Hidden_sigma   : MLE estimate of population std dev (sigma_hat)
#   Cutoff         : Published admission cutoff (c)
#   Accept_Rate    : Implied acceptance rate P(X >= c) from fitted parameters
#   Selection_Bias : Observed mean - Hidden_mu (truncation bias)
# ==============================================================================

# Compile all tier-level results into a list of dictionaries
summary_rows = []
for school_name, results in ALL_MLE_RESULTS.items():
    for tier_num, fitted in results.items():
        summary_rows.append({
            'School': school_name,
            'Tier': tier_num,
            'Hidden_mu': fitted.mu,              # mu_hat: recovered population mean
            'Hidden_sigma': fitted.sigma,        # sigma_hat: recovered population std dev
            'Cutoff': fitted.cutoff,             # c: published cutoff
            'Accept_Rate': fitted.fitted_accept_rate,  # P(X >= c) from fitted params
            'Selection_Bias': fitted.selection_bias,   # X_bar_obs - mu_hat
        })

# Create DataFrame
mle_summary = pd.DataFrame(summary_rows)

# Filter to Tier 4 and sort by hidden mean (most competitive schools first)
t4_summary = mle_summary[mle_summary['Tier'] == 4].sort_values('Hidden_mu', ascending=False)

# Display formatted table
print("=" * 80)
print("TIER 4 MLE RESULTS - Sorted by Hidden Population Mean (mu_hat)")
print("=" * 80)
print()
print(f"{'School':<20} {'mu_hat':>10} {'sigma_hat':>12} {'Cutoff':>10} {'Selection Bias':>15}")
print("-" * 80)

for _, row in t4_summary.iterrows():
    print(f"{row['School']:<20} {row['Hidden_mu']:>10.1f} {row['Hidden_sigma']:>12.1f} "
          f"{row['Cutoff']:>10.1f} {row['Selection_Bias']:>15.1f}")

print()
print("INTERPRETATION:")
print("  - Small sigma_hat (< 30): Extremely competitive tier; scores tightly clustered")
print("  - Large sigma_hat (> 60): Heterogeneous applicant pool; wide score range")
print("  - Selection Bias: How much truncation inflates the published average")

## 5. Visualization: MLE Analysis

### 5.1 Individual School Plots

For each school, I generate a 6-panel visualization to communicate the MLE results:

| Panel | Content | Purpose |
|-------|---------|---------|
| (0,0) | All tier distributions | Overlaid PDFs of $\mathcal{N}(\hat{\mu}, \hat{\sigma}^2)$ for each tier, with cutoff lines |
| (0,1) | Selection bias by tier | Bar chart of $\bar{X}_{\text{obs}} - \hat{\mu}$ |
| (0,2) | Acceptance regions | T1 vs T4 distributions with admitted regions shaded |
| (1,0) | Joint constraint curves | Contour plot showing how mean and acceptance rate constraints intersect |
| (1,1) | True mean vs cutoff | Side-by-side bars showing the gap between $\hat{\mu}$ and $c$ |
| (1,2) | Parameter summary | Key numbers in text format |

The joint constraint plot (panel 1,0) is particularly illuminating: it shows two curves in $(\mu, \sigma)$ space:
- **Blue curve**: All $(\mu, \sigma)$ pairs that produce the observed truncated mean
- **Red curve**: All $(\mu, \sigma)$ pairs that produce the target acceptance rate

The MLE solution is the intersection of these curves. With only one curve (one constraint), the problem is underidentified.

In [ ]:
# ==============================================================================
# VISUALIZATION FUNCTION: INDIVIDUAL SCHOOL MLE PLOTS
# ==============================================================================
# This function generates a comprehensive 6-panel visualization for each school,
# showing the MLE results and how the constraints identify the parameters.
#
# Variables (LaTeX correspondence):
#   mu, sigma     -> mu_hat, sigma_hat : recovered population parameters
#   cutoff        -> c                 : admission threshold
#   t1, t4        -> FittedTier objects for Tier 1 and Tier 4
#   x             -> domain for plotting PDFs (score values 300-920)
#   y             -> f(x; mu, sigma) = normal PDF evaluated at x
#   Z_mean        -> E[X|X>=c] surface over (mu, sigma) grid
#   Z_accept      -> P(X>=c) surface over (mu, sigma) grid
#   tier_colors   -> color palette: T1=red, T2=blue, T3=green, T4=purple
# ==============================================================================

def create_mle_visualization(school_name: str, 
                             results: Dict[int, FittedTier], 
                             school_info: dict) -> plt.Figure:
    """
    Generate comprehensive 6-panel MLE visualization for a single school.
    
    Panel layout (2x3 grid):
        [0,0] All tier distributions    [0,1] Selection bias bars    [0,2] Acceptance regions
        [1,0] Joint constraints         [1,1] Mean vs cutoff         [1,2] Summary text
    
    Parameters
    ----------
    school_name : str
        Name of the school (for plot titles)
    results : Dict[int, FittedTier]
        MLE results by tier (keys: 1, 2, 3, 4)
    school_info : dict
        Original school data (used for constraint curve plotting)
    
    Returns
    -------
    matplotlib.figure.Figure
        The generated 6-panel figure
    """
    # Create 2x3 subplot grid
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Color palette for tiers (colorblind-friendly)
    # T1: Red (disadvantaged), T4: Purple (affluent)
    tier_colors = {1: '#e41a1c', 2: '#377eb8', 3: '#4daf4a', 4: '#984ea3'}
    
    # =========================================================================
    # PANEL [0,0]: All tier distributions overlaid
    # =========================================================================
    # Shows the recovered N(mu_hat, sigma_hat^2) for each tier.
    # Vertical dashed lines indicate cutoffs.
    ax = axes[0, 0]
    x = np.linspace(300, 920, 500)  # Score domain for PDF plotting
    
    for tier in [1, 2, 3, 4]:
        t = results[tier]
        # Compute normal PDF: f(x) = phi((x - mu) / sigma) / sigma
        y = stats.norm.pdf(x, t.mu, t.sigma)
        ax.plot(x, y, color=tier_colors[tier], linewidth=2, 
                label=f'T{tier}: mu={t.mu:.0f}, sigma={t.sigma:.0f}')
        # Vertical line at cutoff c
        ax.axvline(t.cutoff, color=tier_colors[tier], linestyle='--', alpha=0.5)
    
    ax.set_xlabel('Score')
    ax.set_ylabel('Probability Density')
    ax.set_title(f'{school_name}: Recovered Population Distributions', fontweight='bold')
    ax.legend(loc='upper left', fontsize=8)
    ax.set_xlim(300, 920)
    
    # =========================================================================
    # PANEL [0,1]: Selection bias by tier
    # =========================================================================
    # Bar chart showing (X_bar_obs - mu_hat) for each tier.
    # Larger bars indicate more severe truncation (cutoff far above mean).
    ax = axes[0, 1]
    tiers = [1, 2, 3, 4]
    biases = [results[t].selection_bias for t in tiers]
    
    bars = ax.bar(range(4), biases, color=[tier_colors[t] for t in tiers])
    ax.set_xticks(range(4))
    ax.set_xticklabels(['Tier 1', 'Tier 2', 'Tier 3', 'Tier 4'])
    ax.set_ylabel('Selection Bias (points)')
    ax.set_title('Selection Bias by Tier\n(Observed Mean - True Mean)', fontweight='bold')
    
    # Add value labels on top of bars
    for bar, bias in zip(bars, biases):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{bias:.0f}', ha='center', fontweight='bold', fontsize=9)
    
    # =========================================================================
    # PANEL [0,2]: Acceptance regions (T1 vs T4)
    # =========================================================================
    # Compares the full distribution (light shading) with the admitted region
    # (dark shading, X >= c). Illustrates how truncation affects each tier differently.
    ax = axes[0, 2]
    x = np.linspace(300, 920, 500)
    
    t1, t4 = results[1], results[4]
    y1 = stats.norm.pdf(x, t1.mu, t1.sigma)
    y4 = stats.norm.pdf(x, t4.mu, t4.sigma)
    
    # Full distribution (light shading)
    ax.fill_between(x, y1, alpha=0.2, color=tier_colors[1])
    ax.fill_between(x, y4, alpha=0.2, color=tier_colors[4])
    
    # Admitted region only (dark shading): X >= cutoff
    mask1 = x >= t1.cutoff
    mask4 = x >= t4.cutoff
    ax.fill_between(x[mask1], y1[mask1], alpha=0.6, color=tier_colors[1], label='T1 admitted')
    ax.fill_between(x[mask4], y4[mask4], alpha=0.6, color=tier_colors[4], label='T4 admitted')
    
    ax.set_xlabel('Score')
    ax.set_ylabel('Density')
    ax.set_title('Who Gets Admitted?\nLight = Full Population, Dark = Admitted', fontweight='bold')
    ax.legend()
    
    # =========================================================================
    # PANEL [1,0]: Joint constraint curves for T1
    # =========================================================================
    # This panel shows the identification strategy. Two curves in (mu, sigma) space:
    #   Blue: All (mu, sigma) pairs giving the observed truncated mean
    #   Red:  All (mu, sigma) pairs giving the target acceptance rate
    # The MLE solution is their intersection.
    ax = axes[1, 0]
    
    # Create grid over (mu, sigma) parameter space
    mu_range = np.linspace(300, 800, 80)
    sigma_range = np.linspace(30, 200, 80)
    MU, SIGMA = np.meshgrid(mu_range, sigma_range)
    
    # Get T1 data for constraint computation
    t1_data = school_info['tiers'][1]
    tier_applicants = school_info['total_applicants'] / 4
    tier_seats = school_info['seats'] * 0.175
    target_accept = tier_seats / tier_applicants  # r = seats / applicants
    
    # Compute constraint surfaces over the grid
    # Z_mean[i,j] = E[X|X>=c] for (mu, sigma) = (MU[i,j], SIGMA[i,j])
    # Z_accept[i,j] = P(X>=c) for (mu, sigma) = (MU[i,j], SIGMA[i,j])
    Z_mean = np.zeros_like(MU)
    Z_accept = np.zeros_like(MU)
    
    for i in range(MU.shape[0]):
        for j in range(MU.shape[1]):
            Z_mean[i,j] = truncated_mean(MU[i,j], SIGMA[i,j], t1_data['cutoff'])
            Z_accept[i,j] = acceptance_prob(MU[i,j], SIGMA[i,j], t1_data['cutoff'])
    
    # Plot constraint contours
    # Blue: mean constraint (E[X|X>=c] = X_bar_obs)
    ax.contour(MU, SIGMA, Z_mean, levels=[t1_data['avg']], colors='blue', linewidths=2)
    # Red: acceptance rate constraint (P(X>=c) = r)
    ax.contour(MU, SIGMA, Z_accept, levels=[target_accept], colors='red', linewidths=2)
    
    # Mark the MLE solution (intersection point)
    t1_fit = results[1]
    ax.scatter([t1_fit.mu], [t1_fit.sigma], color='purple', s=100, zorder=5, marker='*')
    ax.annotate(f'MLE\n({t1_fit.mu:.0f}, {t1_fit.sigma:.0f})', 
                (t1_fit.mu, t1_fit.sigma), 
                textcoords='offset points', xytext=(10, 10), 
                fontweight='bold', color='purple')
    
    ax.set_xlabel('Hidden Mean (mu)')
    ax.set_ylabel('Hidden Std Dev (sigma)')
    ax.set_title(f'T1 Joint Constraints\nBlue: Mean={t1_data["avg"]:.0f}, Red: Accept={target_accept:.1%}', 
                 fontweight='bold')
    
    # =========================================================================
    # PANEL [1,1]: True mean vs cutoff comparison
    # =========================================================================
    # Side-by-side bars showing mu_hat (what we recovered) vs c (published cutoff).
    # The gap between them is the "headroom" for admitted students.
    ax = axes[1, 1]
    
    true_means = [results[t].mu for t in tiers]
    cutoffs = [results[t].cutoff for t in tiers]
    
    x_pos = np.arange(4)
    width = 0.35
    
    ax.bar(x_pos - width/2, true_means, width, label='True Mean (mu)', 
           color=[tier_colors[t] for t in tiers], alpha=0.8)
    ax.bar(x_pos + width/2, cutoffs, width, label='Cutoff (c)', 
           color=[tier_colors[t] for t in tiers], alpha=0.3, hatch='//')
    
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Tier 1', 'Tier 2', 'Tier 3', 'Tier 4'])
    ax.set_ylabel('Score')
    ax.set_title('True Population Means vs Cutoffs by Tier', fontweight='bold')
    ax.legend()
    
    # =========================================================================
    # PANEL [1,2]: Summary statistics (text box)
    # =========================================================================
    ax = axes[1, 2]
    ax.axis('off')
    
    summary_text = f"""
    {school_name.upper()}
    {'=' * 35}
    
    Tier 1:  mu = {results[1].mu:6.0f},  sigma = {results[1].sigma:5.0f}
    Tier 2:  mu = {results[2].mu:6.0f},  sigma = {results[2].sigma:5.0f}
    Tier 3:  mu = {results[3].mu:6.0f},  sigma = {results[3].sigma:5.0f}
    Tier 4:  mu = {results[4].mu:6.0f},  sigma = {results[4].sigma:5.0f}
    
    {'=' * 35}
    KEY INSIGHTS:
    
    T4-T1 mean gap: {results[4].mu - results[1].mu:.0f} pts
    T1 selection bias: {results[1].selection_bias:.0f} pts
    T4 selection bias: {results[4].selection_bias:.0f} pts
    T4 sigma = {results[4].sigma:.0f} (smaller = more competitive)
    """
    
    ax.text(0.1, 0.9, summary_text, transform=ax.transAxes, 
            fontsize=10, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    return fig

In [ ]:
# ==============================================================================
# GENERATE MLE PLOTS FOR SELECTED SCHOOLS
# ==============================================================================
# I generate detailed plots for a representative sample of schools:
#   - 2 elite schools (Lane Tech, Payton) to show tight T4 distributions
#   - 2 regional schools (Northside, South Shore) to show wider distributions
#
# Generating plots for all 11 schools would be redundant; these four capture
# the key patterns in the data.
# ==============================================================================

# Select representative schools spanning the competitiveness spectrum
schools_to_plot = ['Lane Tech', 'Walter Payton', 'Northside', 'South Shore']

for school_name in schools_to_plot:
    print(f"Generating MLE visualization for {school_name}...")
    
    # Create the 6-panel figure
    fig = create_mle_visualization(
        school_name, 
        ALL_MLE_RESULTS[school_name], 
        SCHOOL_DATA[school_name]
    )
    
    plt.show()
    print()  # Blank line between figures

### 5.2 Combined Density Plots: All Schools

The following visualization shows the recovered population distributions for all 11 schools on a single page, arranged by competitiveness (T4 cutoff descending). This reveals the fundamental **bifurcation** in the SEHS system:

- **Elite schools** (Payton, Northside, Young, Jones, Lane): High $\hat{\mu}$, tight $\hat{\sigma}$, cutoffs near or above 850
- **Regional schools** (Hancock through South Shore): Lower $\hat{\mu}$, wider $\hat{\sigma}$, cutoffs spanning 500-800

The visual contrast is striking: elite school distributions are narrow peaks concentrated near 900, while regional school distributions are broad curves spanning hundreds of points.

In [ ]:
# ==============================================================================
# COMBINED DENSITY PLOT: ALL 11 SCHOOLS
# ==============================================================================
# This function creates a grid of density plots showing all schools on one page.
# Schools are sorted by T4 cutoff (most competitive first).
#
# Variables:
#   school_order   : List of school names sorted by T4 cutoff (descending)
#   tier_colors    : Color palette for tiers
#   x              : Score domain for PDF evaluation
#   y              : Normal PDF values at x
# ==============================================================================

def create_combined_density_plot() -> plt.Figure:
    """
    Create a 3x4 grid of density plots for all 11 schools.
    
    Each subplot shows the recovered N(mu_hat, sigma_hat^2) distributions
    for all four tiers, with cutoff lines overlaid.
    
    Schools are sorted by T4 cutoff (most competitive first), so elite
    schools appear in the top rows and regional schools in the bottom.
    
    Returns
    -------
    matplotlib.figure.Figure
        The combined figure with 11 subplots (12th cell empty)
    """
    # Sort schools by T4 cutoff (descending = most competitive first)
    school_order = sorted(
        SCHOOL_DATA.keys(), 
        key=lambda s: SCHOOL_DATA[s]['tiers'][4]['cutoff'], 
        reverse=True
    )
    
    # Create 3x4 grid (11 schools + 1 empty cell)
    fig, axes = plt.subplots(3, 4, figsize=(20, 15))
    axes = axes.flatten()
    
    # Color scheme for tiers
    tier_colors = {1: '#e41a1c', 2: '#377eb8', 3: '#4daf4a', 4: '#984ea3'}
    
    # Score domain for PDF plotting
    x = np.linspace(300, 920, 500)
    
    for idx, school_name in enumerate(school_order):
        ax = axes[idx]
        results = ALL_MLE_RESULTS[school_name]
        
        # Plot each tier's recovered distribution
        for tier in [1, 2, 3, 4]:
            t = results[tier]
            # Normal PDF: f(x; mu, sigma)
            y = stats.norm.pdf(x, t.mu, t.sigma)
            # Filled area under curve
            ax.fill_between(x, y, alpha=0.3, color=tier_colors[tier])
            # Outline
            ax.plot(x, y, color=tier_colors[tier], linewidth=1.5, label=f'T{tier}')
            # Cutoff line
            ax.axvline(t.cutoff, color=tier_colors[tier], linestyle='--', alpha=0.5, linewidth=1)
        
        ax.set_title(school_name, fontweight='bold', fontsize=11)
        ax.set_xlim(300, 920)
        ax.set_xlabel('Score', fontsize=9)
        ax.set_ylabel('Density', fontsize=9)
        
        # Add legend only to first subplot (to avoid clutter)
        if idx == 0:
            ax.legend(loc='upper left', fontsize=8)
    
    # Hide the unused 12th subplot
    axes[-1].axis('off')
    
    plt.suptitle(
        'MLE-Recovered Population Distributions for All SEHS Schools\n'
        '(Sorted by T4 cutoff; dashed lines indicate cutoffs)', 
        fontsize=14, fontweight='bold', y=1.02
    )
    plt.tight_layout()
    return fig


# Generate and display the combined plot
fig = create_combined_density_plot()
plt.show()

In [ ]:
# ==============================================================================
# TIER 4 COMPARISON: ELITE VS REGIONAL SCHOOLS
# ==============================================================================
# This visualization directly compares T4 distributions between school groups.
# The contrast reveals the bifurcation in the SEHS system.
#
# Left panel:  Elite schools (Payton, Northside, Young, Jones, Lane)
#              - sigma_hat typically 10-30 (very tight)
#              - mu_hat typically 830-890 (near ceiling)
#
# Right panel: Regional schools (Hancock, Lindblom, Brooks, Westinghouse, King, South Shore)
#              - sigma_hat typically 60-100 (wide spread)
#              - mu_hat typically 590-720 (much lower)
# ==============================================================================

def create_tier_comparison_plot() -> plt.Figure:
    """
    Compare T4 distributions between elite and regional schools.
    
    This visualization highlights the fundamental difference in T4 competition:
    - Elite schools: sigma ~ 10-30 (extreme competition, ceiling effects)
    - Regional schools: sigma ~ 60-100 (more heterogeneous applicant pools)
    
    Returns
    -------
    matplotlib.figure.Figure
        Two-panel figure comparing T4 distributions
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Define school groups based on competitiveness
    elite_schools = ['Walter Payton', 'Northside', 'Whitney Young', 'Jones', 'Lane Tech']
    regional_schools = ['Lindblom', 'Hancock', 'Brooks', 'King', 'Westinghouse', 'South Shore']
    
    # Score domain for PDF plotting
    x = np.linspace(400, 920, 500)
    
    # -------------------------------------------------------------------------
    # LEFT PANEL: Elite schools T4
    # -------------------------------------------------------------------------
    ax = axes[0]
    # Use a blue color gradient (light to dark)
    colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(elite_schools)))
    
    for i, school in enumerate(elite_schools):
        t = ALL_MLE_RESULTS[school][4]  # Get T4 results
        y = stats.norm.pdf(x, t.mu, t.sigma)
        ax.plot(x, y, color=colors[i], linewidth=2, 
                label=f'{school} (mu={t.mu:.0f}, sigma={t.sigma:.0f})')
        # Cutoff line
        ax.axvline(t.cutoff, color=colors[i], linestyle='--', alpha=0.5)
    
    ax.set_xlabel('Score', fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title('Tier 4 Distributions: Elite Schools\n(Tight sigma indicates extreme competition)', 
                 fontweight='bold')
    ax.legend(loc='upper left', fontsize=9)
    ax.set_xlim(600, 920)
    
    # -------------------------------------------------------------------------
    # RIGHT PANEL: Regional schools T4
    # -------------------------------------------------------------------------
    ax = axes[1]
    # Use a red color gradient (light to dark)
    colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(regional_schools)))
    
    for i, school in enumerate(regional_schools):
        t = ALL_MLE_RESULTS[school][4]  # Get T4 results
        y = stats.norm.pdf(x, t.mu, t.sigma)
        ax.plot(x, y, color=colors[i], linewidth=2, 
                label=f'{school} (mu={t.mu:.0f}, sigma={t.sigma:.0f})')
        # Cutoff line
        ax.axvline(t.cutoff, color=colors[i], linestyle='--', alpha=0.5)
    
    ax.set_xlabel('Score', fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title('Tier 4 Distributions: Regional Schools\n(Wide sigma reflects heterogeneous pools)', 
                 fontweight='bold')
    ax.legend(loc='upper left', fontsize=9)
    ax.set_xlim(400, 900)
    
    plt.tight_layout()
    return fig


# Generate and display the comparison plot
fig = create_tier_comparison_plot()
plt.show()

## 6. Physics-Based Monte Carlo Simulation (v13)

### 6.1 Motivation and Approach

The MLE analysis recovers population-level parameters $(\hat{\mu}, \hat{\sigma})$ for each school-tier combination, but it does not model the **behavioral dynamics** of how students choose schools. To simulate the full admissions process and predict cutoffs, I need:

1. A model of how students form **preferences** over schools
2. A mechanism for generating **realistic score distributions** by region and tier
3. An implementation of the **serial dictatorship matching algorithm**

I call this a "physics-based" simulation because it models students as agents with utility functions, analogous to particles in a potential field. Each student-school pair has an associated utility, and students rank schools by utility (highest to lowest).

### 6.2 Model Components

**Utility function**: For student $i$ considering school $j$:

$$U_{ij} = P_j - d_{ij} \cdot f(t_i, s_i) - \mathbb{1}[r_i \neq r_j] \cdot \gamma(r_i, r_j, s_i) + \delta_j(t_i, s_i, r_i)$$

| Term | Meaning |
|------|---------|
| $P_j$ | Prestige of school $j$ (0-100 scale, based on cutoffs and reputation) |
| $d_{ij}$ | Distance in miles from student $i$'s home to school $j$ |
| $f(t_i, s_i)$ | Friction coefficient (varies by tier $t_i$ and score $s_i$; high scorers travel more) |
| $\gamma(r_i, r_j, s_i)$ | Cross-region penalty (students prefer local schools; penalty depends on region pair) |
| $\delta_j(\cdot)$ | School-specific demand modifier (tuned via Optuna to match observed cutoffs) |

**Score generation**: I use skewed normal distributions:

$$X_{r,t} \sim \text{SkewNorm}(\alpha_{r,t}, \mu_{r,t}, \sigma_{r,t})$$

where parameters vary by region $r \in \{\text{north}, \text{loop}, \text{west}, \text{south}\}$ and tier $t \in \{1,2,3,4\}$.

**Justification for skewed distributions**: The pure normal assumption from MLE is a simplification. Empirically, score distributions exhibit:
- **Positive skew in lower tiers**: Long right tail of high achievers who beat the odds
- **Negative skew in higher tiers**: Compression against the 900 ceiling

The skewnorm distribution captures this asymmetry with a single additional parameter $\alpha$ (skewness).

In [ ]:
# ==============================================================================
# IMPORT CENTRALIZED DATA FROM sehs_data.py
# ==============================================================================
# The sehs_data.py module contains all school configurations, geographic data,
# and historical cutoffs. This separation keeps the simulation code clean and
# allows easy updates when new data becomes available.
#
# Key imports:
#   SCHOOLS                  : Dict of School objects (prestige, location, seats)
#   REGIONS                  : Geographic region definitions (center, spread)
#   TIER_BY_REGION           : Probability of each tier in each region
#   ADMISSIONS               : Seat allocation rules (30% rank, 70% tier)
#   ELITE_SCHOOLS            : List of top 5 schools by prestige
#   CUTOFFS_2024_CALIBRATION : Actual cutoffs for model validation
#   distance_miles           : Haversine distance function
# ==============================================================================

import sys
sys.path.insert(0, '/Users/andrewhood/Tutoring/School Modeling')

from sehs_data import (
    SCHOOLS,                    # School objects with prestige, location, seats
    REGIONS,                    # Geographic region definitions
    TIER_BY_REGION,            # Distribution of tiers across regions
    ADMISSIONS,                # Seat allocation rules (30/70 split)
    ELITE_SCHOOLS,             # List of top 5 schools
    CUTOFFS_2024_CALIBRATION,  # Actual cutoffs for validation
    distance_miles             # Haversine distance calculation
)

print(f"Loaded {len(SCHOOLS)} schools from sehs_data.py")
print(f"Elite schools: {', '.join(ELITE_SCHOOLS)}")
print(f"Regions: {', '.join(REGIONS.keys())}")

In [ ]:
# ==============================================================================
# V13 MODEL PARAMETERS (OPTUNA-TUNED)
# ==============================================================================
# These parameters were optimized via 500 trials of Bayesian hyperparameter
# tuning using the Optuna framework (Tree-structured Parzen Estimator sampler).
#
# Objective: Minimize MAE between simulated and actual cutoffs, subject to:
#   - Max error < 80 points (any single tier at any school)
#   - Max school MAE < 25 points (worst school's average error)
#
# Best trial: #353
#   MAE = 22.79, Max Error = 84.4, Max School MAE = 30.0
#
# Variables (LaTeX correspondence):
#   loc   -> mu_{r,t}    : location parameter of skewnorm (related to mean)
#   scale -> sigma_{r,t} : scale parameter of skewnorm (related to std dev)
#   skew  -> alpha_{r,t} : shape parameter (>0 right skew, <0 left skew)
# ==============================================================================

# -----------------------------------------------------------------------------
# SCORE DISTRIBUTIONS BY REGION AND TIER
# -----------------------------------------------------------------------------
# These define the generative model for student scores.
# Parameters are for scipy.stats.skewnorm(a, loc, scale):
#   a (skew): Shape parameter controlling asymmetry
#   loc:      Location parameter (shifts the distribution)
#   scale:    Scale parameter (stretches the distribution)
#
# North/Loop: High scores, left-skewed T4 (ceiling effects)
# West/South: Lower scores, right-skewed (long tail of high achievers)
# -----------------------------------------------------------------------------

V13_SCORE_DISTRIBUTIONS = {
    # North Side: Serves elite schools (Payton, Northside, Lane Tech)
    # T4 is tightly concentrated near 870 because Northside/Payton T4 cutoffs are 893-898
    'north': {
        1: {'loc': 620, 'scale': 100, 'skew': 3.0},   # Right-skewed: long tail of high achievers
        2: {'loc': 720, 'scale': 80, 'skew': 1.5},    # Moderate right skew
        3: {'loc': 830, 'scale': 45, 'skew': -2.0},   # Left-skewed: ceiling effects
        4: {'loc': 870, 'scale': 22, 'skew': -3.5},   # Very tight, left-skewed (near 900 ceiling)
    },
    
    # Loop (Downtown): Similar to North, serves Whitney Young, Jones
    # Slightly higher T1 due to magnet school feeder effects
    'loop': {
        1: {'loc': 700, 'scale': 90, 'skew': 2.5},
        2: {'loc': 755, 'scale': 75, 'skew': 1.0},
        3: {'loc': 835, 'scale': 40, 'skew': -2.0},
        4: {'loc': 872, 'scale': 20, 'skew': -3.5},
    },
    
    # West Side: Lower scores, serves Westinghouse primarily
    # OPTUNA-TUNED (500 trials, trial 353, MAE 22.79)
    # These values were found via Bayesian optimization
    'west': {
        1: {'loc': 453, 'scale': 113, 'skew': 4.77},  # Strong right skew (few high achievers)
        2: {'loc': 558, 'scale': 90, 'skew': 2.20},
        3: {'loc': 592, 'scale': 71, 'skew': 2.13},
        4: {'loc': 689, 'scale': 51, 'skew': 1.32},   # Still right-skewed (unlike North T4)
    },
    
    # South Side: Lower scores, serves King/Brooks/South Shore/Lindblom/Hancock
    # OPTUNA-TUNED (500 trials, trial 353, MAE 22.79)
    'south': {
        1: {'loc': 457, 'scale': 100, 'skew': 4.04},
        2: {'loc': 541, 'scale': 87, 'skew': 3.31},
        3: {'loc': 612, 'scale': 79, 'skew': 0.58},   # Nearly symmetric
        4: {'loc': 724, 'scale': 47, 'skew': 0.96},   # Slight right skew
    },
}

# -----------------------------------------------------------------------------
# BEHAVIORAL PARAMETERS
# -----------------------------------------------------------------------------
# These control how students form preferences and make decisions.
# -----------------------------------------------------------------------------

PARAMS = {
    # Total number of students to generate (approximate SEHS applicant pool)
    'n_students': 22000,
    
    # Distance friction by tier: utility cost per mile
    # Lower tiers are more distance-sensitive (less willing to commute far)
    # Rationale: T1 families may have less flexible transportation
    'base_friction': {1: 3.0, 2: 2.2, 3: 1.5, 4: 1.0},
    
    # Score-based mobility: high scorers are more willing to travel
    # This captures the observation that top students target elite schools
    # regardless of distance (they know they can get in anywhere)
    'score_mobility': {
        'threshold_high': 850,    # Above 850: very mobile
        'threshold_mid': 750,     # 750-850: moderately mobile
        'multiplier_high': 0.3,   # 70% friction reduction for 850+ scorers
        'multiplier_mid': 0.6,    # 40% friction reduction for 750-849 scorers
        'multiplier_low': 1.0,    # No reduction for <750 scorers
    },
    
    # Cross-region penalties (asymmetric by region pair)
    # Students generally prefer schools in their home region
    # North-to-South penalty is highest (affluent families avoid South Side)
    # South-to-North penalty is lower (elite schools are worth the commute)
    'cross_region_base': {
        ('north', 'south'): 50,   # North students strongly avoid South schools
        ('north', 'west'): 35,
        ('south', 'north'): 25,   # South students less penalty (elite schools worth it)
        ('south', 'west'): 20,
        ('south', 'loop'): 5,     # Loop accessible from South via CTA
        ('west', 'north'): 30,
        ('west', 'south'): 25,
        ('west', 'loop'): 10,
        ('loop', 'south'): 15,
        ('loop', 'north'): 8,
        ('loop', 'west'): 12,
    },
    
    # Private school exit: high-scoring T4 students leave the CPS pool
    # Models families who choose private schools or move to suburbs
    # Higher exit rates in West/South reflect "brain drain" to private schools
    'exit_threshold': {'north': 892, 'loop': 894, 'west': 820, 'south': 800},
    'exit_prob': {'north': 0.18, 'loop': 0.22, 'west': 0.35, 'south': 0.40},
    
    # Maximum distance a student will consider (miles)
    # Beyond this, utility drops to -infinity (school not ranked)
    'distance_cap': 15.0,
}

# -----------------------------------------------------------------------------
# SCHOOL-SPECIFIC DEMAND MODIFIERS (OPTUNA-TUNED)
# -----------------------------------------------------------------------------
# These adjust utility for specific schools to match observed demand patterns.
# Negative values reduce demand (fewer students rank the school highly).
# Tuned via Optuna to minimize prediction error.
#
# Key patterns:
#   - King, Hancock, Brooks T4 have large negative penalties (-24, -24, -3)
#     because high-scoring T4 students prefer elite schools
#   - Westinghouse has -36 penalty for non-West students (very local draw)
#   - Lindblom T4 has small penalty (-3) because it competes with South schools
# -----------------------------------------------------------------------------

DEMAND_PENALTIES = {
    'king_t4': -24,           # King T4 has low demand (elite school competition)
    'king_other': -12,        # Other tiers also reduced
    'hancock_t4': -24,        # Hancock T4 similar pattern
    'hancock_other': 5,       # But T1-T3 have slight boost (local preference)
    'brooks_t4': -3,          # Brooks T4 less penalized
    'brooks_other': -12,
    'westinghouse_non_west': -36,  # Non-West students rarely apply
    'westinghouse_t34': -4,
    'south_shore_t4': -10,
    'south_shore_other': -9,
    'lindblom_t4': -3,
}

print("v13 model parameters loaded.")
print(f"Student population: {PARAMS['n_students']:,}")
print(f"Regions: North, Loop, West, South")

In [ ]:
# ==============================================================================
# STUDENT GENERATION FUNCTION
# ==============================================================================
# This function creates a synthetic population of ~22,000 students with:
#   - Tier assignment (1-4) based on citywide tier composition
#   - Region assignment based on tier-specific regional probabilities
#   - Geographic location (lat, lon) drawn from region-specific Gaussian
#   - Score drawn from region x tier-specific skewed normal distribution
#   - Private school exit applied to high-scoring T4 students
#
# Variables (LaTeX correspondence):
#   n         -> N              : total number of students to generate
#   tiers     -> t_i            : tier assignment for student i
#   regions   -> r_i            : region assignment for student i
#   scores    -> s_i            : composite score for student i
#   lats/lons -> (lat_i, lon_i) : geographic coordinates for student i
#
# The generation follows a hierarchical model:
#   1. t_i ~ Categorical(0.25, 0.25, 0.25, 0.25)  [uniform tier assignment]
#   2. r_i | t_i ~ Categorical(p_{t_i})           [tier-dependent region]
#   3. (lat_i, lon_i) | r_i ~ N(center_{r_i}, Sigma_{r_i})
#   4. s_i | r_i, t_i ~ SkewNorm(alpha_{r,t}, mu_{r,t}, sigma_{r,t})
# ==============================================================================

def generate_students(n: int = 22000, seed: int = None) -> pd.DataFrame:
    """
    Generate a synthetic student population with realistic characteristics.
    
    The generation process:
    1. Assign tier (1-4) uniformly (CPS assigns based on census tract)
    2. Assign region based on tier-specific probabilities (T4 more in North)
    3. Generate location from region-specific bivariate Gaussian
    4. Generate score from region x tier skewed normal distribution
    5. Apply private school exit (remove some high-scoring T4 students)
    
    Parameters
    ----------
    n : int, default=22000
        Number of students to generate (approximate SEHS applicant pool size)
    seed : int or None
        Random seed for reproducibility. If None, uses current RNG state.
    
    Returns
    -------
    pd.DataFrame
        DataFrame with columns:
        - Tier: int (1-4)
        - Region: str ('north', 'loop', 'west', 'south')
        - Score: float (400-900, clipped)
        - Lat, Lon: float (geographic coordinates)
        - TieBreaker: float (for breaking score ties in matching)
    """
    if seed is not None:
        np.random.seed(seed)
    
    # -------------------------------------------------------------------------
    # STEP 1: Assign tiers uniformly
    # -------------------------------------------------------------------------
    # CPS assigns tiers based on census tract socioeconomic characteristics.
    # I assume approximately equal citywide representation in each tier.
    # This is a simplification; in reality, tier sizes may differ slightly.
    tiers = np.random.choice([1, 2, 3, 4], size=n, p=[0.25, 0.25, 0.25, 0.25])
    
    # -------------------------------------------------------------------------
    # STEP 2: Assign regions based on tier
    # -------------------------------------------------------------------------
    # Higher tiers (T3, T4) are more concentrated in North/Loop (affluent areas).
    # Lower tiers (T1, T2) are more concentrated in West/South (lower-income areas).
    # TIER_BY_REGION contains these conditional probabilities P(region | tier).
    regions = np.empty(n, dtype='<U10')
    
    for tier in [1, 2, 3, 4]:
        tier_mask = (tiers == tier)
        n_tier = np.sum(tier_mask)
        
        # Get conditional distribution P(region | tier)
        region_probs = TIER_BY_REGION[tier]
        region_names = list(region_probs.keys())
        region_weights = [region_probs[r] for r in region_names]
        
        # Sample regions for this tier
        regions[tier_mask] = np.random.choice(
            region_names, size=n_tier, p=region_weights
        )
    
    # -------------------------------------------------------------------------
    # STEP 3: Generate geographic locations
    # -------------------------------------------------------------------------
    # Each region has a center (lat_center, lon_center) and spread (lat_std, lon_std).
    # Student locations are drawn from a bivariate Gaussian centered on the region.
    lats = np.zeros(n)
    lons = np.zeros(n)
    
    for region_name, region_data in REGIONS.items():
        mask = (regions == region_name)
        n_region = np.sum(mask)
        
        if n_region > 0:
            # Draw from N(center, std^2) independently for lat and lon
            lats[mask] = np.random.normal(
                region_data['lat_center'], 
                region_data['lat_std'], 
                n_region
            )
            lons[mask] = np.random.normal(
                region_data['lon_center'], 
                region_data['lon_std'], 
                n_region
            )
    
    # -------------------------------------------------------------------------
    # STEP 4: Generate scores using skewed normal distributions
    # -------------------------------------------------------------------------
    # Scores depend on both region and tier, reflecting:
    #   - Higher scores in North/Loop (better-resourced schools)
    #   - Higher scores in higher tiers (socioeconomic advantage)
    # I use skewed normal to capture asymmetry (right skew in T1, left skew in T4).
    scores = np.zeros(n)
    
    for region_name in REGIONS.keys():
        for tier in [1, 2, 3, 4]:
            mask = (regions == region_name) & (tiers == tier)
            n_group = np.sum(mask)
            
            if n_group > 0:
                # Get distribution parameters for this region x tier
                dist_params = V13_SCORE_DISTRIBUTIONS[region_name][tier]
                
                # Draw from SkewNorm(a=skew, loc=loc, scale=scale)
                # a > 0: right skew (long right tail)
                # a < 0: left skew (long left tail, ceiling effects)
                scores[mask] = skewnorm.rvs(
                    a=dist_params['skew'],      # Shape (skewness) parameter alpha
                    loc=dist_params['loc'],     # Location parameter mu
                    scale=dist_params['scale'], # Scale parameter sigma
                    size=n_group
                )
    
    # Add small noise (measurement error) and clip to valid range [400, 900]
    # The 400 lower bound represents the practical minimum composite score
    scores = np.clip(scores + np.random.normal(0, 3, n), 400, 900)
    
    # -------------------------------------------------------------------------
    # CREATE DATAFRAME
    # -------------------------------------------------------------------------
    df = pd.DataFrame({
        'Tier': tiers,
        'Region': regions,
        'Score': scores,
        'Lat': lats,
        'Lon': lons,
    })
    
    # -------------------------------------------------------------------------
    # STEP 5: Apply private school exit
    # -------------------------------------------------------------------------
    # Very high-scoring T4 students may opt out of CPS entirely:
    #   - Choose private high schools (Latin, Lab, Parker, etc.)
    #   - Move to high-performing suburban districts
    # This is modeled as probabilistic removal above a threshold.
    # Exit rates are higher in West/South (brain drain to private schools).
    for region in REGIONS.keys():
        threshold = PARAMS['exit_threshold'].get(region, 850)
        prob = PARAMS['exit_prob'].get(region, 0.3)
        
        # Find high-scoring T4 students in this region
        candidates = df[
            (df['Region'] == region) & 
            (df['Tier'] == 4) & 
            (df['Score'] > threshold)
        ].index
        
        if len(candidates) > 0:
            # Remove a fraction of them (simulating private school choice)
            n_drop = int(len(candidates) * prob)
            drop_idx = np.random.choice(
                candidates, 
                size=min(n_drop, len(candidates)), 
                replace=False
            )
            df = df.drop(drop_idx)
    
    # Add tiebreaker for matching (random lottery for equal scores)
    df['TieBreaker'] = np.random.random(len(df))
    
    return df.reset_index(drop=True)


# ==============================================================================
# GENERATE SAMPLE POPULATION
# ==============================================================================
students = generate_students(seed=42)

print(f"Generated {len(students):,} students (after private school exit)")
print()
print("Distribution by region:")
print(students.groupby('Region')['Score'].agg(['count', 'mean', 'std']).round(1))

### 6.3 Visualizing Generated Score Distributions

Before running the full simulation, I visualize the generated score distributions to verify they match the modeling assumptions. This serves as a sanity check that the skewnorm parameters produce reasonable score distributions.

Key things to verify:
1. **Tier ordering**: T4 scores should be highest, T1 lowest
2. **Regional differences**: North/Loop should have higher scores than West/South
3. **Skewness patterns**: T1 should be right-skewed, T4 should be left-skewed (for North/Loop)

In [ ]:
# ==============================================================================
# VISUALIZATION: GENERATED SCORE DISTRIBUTIONS
# ==============================================================================
# This function creates a 2x2 grid of plots to visualize the generated student
# population and verify the score distributions match modeling assumptions.
#
# Variables:
#   students    : DataFrame of generated students
#   tier_colors : Color palette for tiers (T1=red, T2=blue, T3=green, T4=purple)
#   region_colors : Color palette for regions (north=blue, loop=orange, west=green, south=red)
#   region_order : Canonical ordering of regions for consistent plotting
# ==============================================================================

def plot_score_distributions(students: pd.DataFrame) -> plt.Figure:
    """
    Visualize generated score distributions by region and tier.
    
    Creates a 2x2 grid:
    - [0,0] Histograms by tier (all regions combined)
    - [0,1] Overlapping density estimates by tier
    - [1,0] T4 distributions by region (critical for elite school calibration)
    - [1,1] Boxplots by region (shows medians and spread)
    
    Parameters
    ----------
    students : pd.DataFrame
        Generated student data with columns: Tier, Region, Score
    
    Returns
    -------
    matplotlib.figure.Figure
        The 2x2 visualization figure
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Color schemes
    tier_colors = {1: '#e41a1c', 2: '#377eb8', 3: '#4daf4a', 4: '#984ea3'}
    region_colors = {'north': '#1f77b4', 'loop': '#ff7f0e', 'west': '#2ca02c', 'south': '#d62728'}
    region_order = ['north', 'loop', 'west', 'south']
    
    # -------------------------------------------------------------------------
    # PANEL [0,0]: Histograms by tier (all regions combined)
    # -------------------------------------------------------------------------
    # Shows the overall score distribution for each tier, aggregating all regions.
    # Expect: T4 highest, T1 lowest; all distributions should overlap substantially.
    ax = axes[0, 0]
    for tier in [1, 2, 3, 4]:
        tier_scores = students[students['Tier'] == tier]['Score']
        ax.hist(tier_scores, bins=50, alpha=0.5, label=f'Tier {tier}', 
                color=tier_colors[tier], density=True)
    
    ax.set_xlabel('Score')
    ax.set_ylabel('Density')
    ax.set_title('Score Distributions by Tier (All Regions)', fontweight='bold')
    ax.legend()
    
    # -------------------------------------------------------------------------
    # PANEL [0,1]: Overlapping density estimates
    # -------------------------------------------------------------------------
    # Shows tier separation more clearly using filled area plots.
    # Plot in reverse order (T4 first) so T1 appears on top.
    ax = axes[0, 1]
    bins = np.linspace(400, 900, 60)
    
    for tier in [4, 3, 2, 1]:  # Reverse order for visual layering
        tier_scores = students[students['Tier'] == tier]['Score']
        counts, bin_edges = np.histogram(tier_scores, bins=bins, density=True)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        ax.fill_between(bin_centers, counts, alpha=0.35, 
                        color=tier_colors[tier], label=f'Tier {tier}')
        ax.plot(bin_centers, counts, color=tier_colors[tier], linewidth=1.5)
    
    ax.set_xlabel('Score')
    ax.set_ylabel('Density')
    ax.set_title('Overlapping Tier Distributions', fontweight='bold')
    ax.legend()
    
    # -------------------------------------------------------------------------
    # PANEL [1,0]: T4 distributions by region
    # -------------------------------------------------------------------------
    # This is critical for calibrating elite school cutoffs.
    # North/Loop T4 should be tightly concentrated near 870.
    # West/South T4 should be wider and lower.
    ax = axes[1, 0]
    t4_students = students[students['Tier'] == 4]
    
    for region in region_order:
        region_scores = t4_students[t4_students['Region'] == region]['Score']
        ax.hist(region_scores, bins=40, alpha=0.5, label=f'{region.title()} T4', 
                color=region_colors[region], density=True)
    
    ax.set_xlabel('Score')
    ax.set_ylabel('Density')
    ax.set_title('Tier 4 Score Distributions by Region', fontweight='bold')
    ax.legend()
    
    # -------------------------------------------------------------------------
    # PANEL [1,1]: Boxplots by region (all tiers)
    # -------------------------------------------------------------------------
    # Shows medians, quartiles, and outliers for each region.
    # Expect: North > Loop > South > West (approximately)
    ax = axes[1, 1]
    data_for_box = [students[students['Region'] == r]['Score'].values 
                    for r in region_order]
    
    bp = ax.boxplot(data_for_box, labels=[r.title() for r in region_order], 
                    patch_artist=True)
    
    # Color the boxes by region
    for patch, region in zip(bp['boxes'], region_order):
        patch.set_facecolor(region_colors[region])
        patch.set_alpha(0.6)
    
    ax.set_xlabel('Region')
    ax.set_ylabel('Score')
    ax.set_title('Score Distribution by Region (All Tiers)', fontweight='bold')
    
    plt.tight_layout()
    return fig


# Generate and display the distribution plots
fig = plot_score_distributions(students)
plt.show()

### 6.4 Running the Full v13 Simulation

I now import and run the complete v13 simulation from `sehs_simulation_v13.py`. The simulation performs:

1. **Student generation**: Creates ~22,000 students with scores, tiers, regions, and locations
2. **Preference calculation**: Each student computes utility for each school and ranks them
3. **Serial dictatorship matching**: Students processed in score order; each assigned to highest-ranked school with seats
4. **Cutoff computation**: For each school-tier, record the score of the last admitted student
5. **Error analysis**: Compare simulated cutoffs to actual 2024-2025 cutoffs

The simulation is stochastic (random student generation), so I run multiple seeds to assess stability.

In [ ]:
# ==============================================================================
# IMPORT AND RUN THE FULL V13 SIMULATION
# ==============================================================================
# The sehs_simulation_v13.py module contains the complete simulation:
#   - generate_students(): Creates synthetic student population
#   - compute_preferences(): Calculates utility-based school rankings
#   - run_matching(): Implements serial dictatorship algorithm
#   - compute_cutoffs(): Extracts cutoffs from matching results
#   - compute_metrics(): Compares to actual cutoffs, computes MAE
#
# The run_simulation() function wraps all of these steps.
# ==============================================================================

from sehs_simulation_v13 import run_simulation, compute_cutoffs, compute_metrics

print("Running v13 simulation with seed=42...")
print("This generates ~22,000 students and matches them to 11 schools.")
print()

# Run the full simulation pipeline
# Returns:
#   matched_students: DataFrame with student data and matched school
#   results_df: DataFrame comparing simulated vs actual cutoffs
#   metrics: Dict with MAE, max_error, school-level errors
matched_students, results_df, metrics = run_simulation(seed=42, verbose=True)

In [ ]:
# ==============================================================================
# MULTI-SEED VALIDATION
# ==============================================================================
# Run the simulation with multiple random seeds to assess stability.
# A good model should produce consistent MAE across different random draws.
#
# I test 5 seeds: 42, 123, 456, 789, 1000
# For each seed, I record MAE and max_error.
# The mean and standard deviation of MAE across seeds indicates model robustness.
#
# If std(MAE) is small (< 2 points), the model is stable.
# If std(MAE) is large, results depend heavily on the random draw.
# ==============================================================================

print("MULTI-SEED VALIDATION")
print("=" * 60)
print("Testing model stability across 5 random seeds...")
print()

validation_results = []
seeds_to_test = [42, 123, 456, 789, 1000]

for seed in seeds_to_test:
    # Run simulation with this seed (verbose=False to suppress output)
    _, _, m = run_simulation(seed=seed, verbose=False)
    
    # Record key metrics
    validation_results.append({
        'seed': seed,
        'mae': m['overall_mae'],
        'max_error': m['max_error'],
    })
    print(f"Seed {seed}: MAE = {m['overall_mae']:.1f}, Max Error = {m['max_error']:.0f}")

# Convert to DataFrame for analysis
val_df = pd.DataFrame(validation_results)

print()
print("-" * 60)
print("SUMMARY STATISTICS:")
print(f"  Mean MAE:       {val_df['mae'].mean():.1f} points")
print(f"  Std Dev MAE:    {val_df['mae'].std():.1f} points")
print(f"  MAE Range:      [{val_df['mae'].min():.1f}, {val_df['mae'].max():.1f}]")
print(f"  Mean Max Error: {val_df['max_error'].mean():.0f} points")
print()
print("INTERPRETATION:")
if val_df['mae'].std() < 2:
    print("  Model is STABLE: MAE varies by < 2 points across seeds.")
else:
    print("  Model shows VARIABILITY: MAE varies by > 2 points across seeds.")

### 6.5 Simulation Results Visualization

I create a 4-panel visualization to analyze the simulation results:

1. **Per-school MAE**: Which schools are hardest to predict?
2. **Simulated vs actual T4 cutoffs**: How well do predictions match reality?
3. **Admits by region**: Do students from each region attend expected schools?
4. **Error distribution**: Are errors symmetric around zero (unbiased)?

In [ ]:
# ==============================================================================
# VISUALIZATION: SIMULATION RESULTS
# ==============================================================================
# This function creates a 2x2 grid analyzing simulation performance.
#
# Variables:
#   metrics         : Dict containing overall_mae, max_error, school_errors
#   matched_students: DataFrame with student data and matched school
#   cutoffs         : Dict[school][tier] -> simulated cutoff score
#   school_errors   : List of (school_name, mae, max_err) tuples
#   errors          : List of (simulated - actual) for all school-tier combinations
# ==============================================================================

def plot_simulation_results(metrics: dict, matched_students: pd.DataFrame) -> plt.Figure:
    """
    Visualize simulation results and comparison to actual cutoffs.
    
    Creates a 2x2 grid:
    - [0,0] Per-school MAE (horizontal bar chart)
    - [0,1] Simulated vs actual T4 cutoffs (scatter plot with identity line)
    - [1,0] Admits by region (elite vs regional schools)
    - [1,1] Distribution of prediction errors (histogram)
    
    Parameters
    ----------
    metrics : dict
        Output from compute_metrics(), containing:
        - overall_mae: float
        - max_error: float
        - school_errors: Dict[school] -> {'mae': float, 'max_err': float}
    matched_students : pd.DataFrame
        Student data with 'Matched' column indicating assigned school
    
    Returns
    -------
    matplotlib.figure.Figure
        The 2x2 results figure
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # -------------------------------------------------------------------------
    # PANEL [0,0]: Per-school MAE (horizontal bar chart)
    # -------------------------------------------------------------------------
    # Shows which schools are hardest to predict.
    # Elite schools (red) typically have lower MAE than regional schools (blue).
    ax = axes[0, 0]
    
    # Extract school errors and sort by MAE (highest first)
    school_errors = [(name, err['mae'], err['max_err']) 
                     for name, err in metrics['school_errors'].items()]
    school_errors.sort(key=lambda x: x[1], reverse=True)
    
    schools = [s[0] for s in school_errors]
    maes = [s[1] for s in school_errors]
    
    # Color elite schools differently
    colors = ['#d62728' if s in ELITE_SCHOOLS else '#1f77b4' for s in schools]
    
    bars = ax.barh(schools, maes, color=colors, alpha=0.7)
    ax.axvline(25, color='red', linestyle='--', linewidth=2, label='Target: MAE < 25')
    ax.set_xlabel('Mean Absolute Error (points)')
    ax.set_title('Per-School MAE\n(Red = Elite Schools, Blue = Regional Schools)', fontweight='bold')
    ax.legend(loc='lower right')
    
    # -------------------------------------------------------------------------
    # PANEL [0,1]: Simulated vs Actual T4 cutoffs (scatter plot)
    # -------------------------------------------------------------------------
    # Points on the diagonal indicate perfect prediction.
    # Deviations from diagonal show systematic over/under-prediction.
    ax = axes[0, 1]
    
    # Compute cutoffs from matched students
    cutoffs = compute_cutoffs(matched_students)
    
    sim_t4 = []
    actual_t4 = []
    school_names = []
    
    for school in SCHOOLS.keys():
        if 4 in cutoffs.get(school, {}) and 4 in CUTOFFS_2024_CALIBRATION.get(school, {}):
            sim_t4.append(cutoffs[school][4])
            actual_t4.append(CUTOFFS_2024_CALIBRATION[school][4])
            school_names.append(school)
    
    ax.scatter(actual_t4, sim_t4, s=100, alpha=0.7)
    
    # Add school name labels
    for i, name in enumerate(school_names):
        ax.annotate(name[:8], (actual_t4[i], sim_t4[i]), fontsize=8, alpha=0.7)
    
    # Add diagonal line (perfect prediction)
    ax.plot([500, 900], [500, 900], 'r--', linewidth=2, label='Perfect fit')
    ax.set_xlabel('Actual T4 Cutoff')
    ax.set_ylabel('Simulated T4 Cutoff')
    ax.set_title('Simulated vs Actual T4 Cutoffs', fontweight='bold')
    ax.legend()
    
    # -------------------------------------------------------------------------
    # PANEL [1,0]: Admits by region (elite vs regional schools)
    # -------------------------------------------------------------------------
    # Shows whether students from each region attend the expected schools.
    # North/Loop students should dominate elite schools.
    # West/South students should appear at regional schools.
    ax = axes[1, 0]
    
    # Filter to matched students only
    matched = matched_students[matched_students['Matched'].notna()]
    
    # Count admits by region and school
    region_school_counts = matched.groupby(['Region', 'Matched']).size().unstack(fill_value=0)
    
    # Aggregate to elite vs regional schools
    elite_cols = [s for s in ELITE_SCHOOLS if s in region_school_counts.columns]
    other_cols = [s for s in region_school_counts.columns if s not in ELITE_SCHOOLS]
    
    elite_counts = region_school_counts[elite_cols].sum(axis=1) if elite_cols else pd.Series(0, index=region_school_counts.index)
    other_counts = region_school_counts[other_cols].sum(axis=1) if other_cols else pd.Series(0, index=region_school_counts.index)
    
    regions = ['north', 'loop', 'west', 'south']
    x = np.arange(len(regions))
    width = 0.35
    
    ax.bar(x - width/2, [elite_counts.get(r, 0) for r in regions], width, 
           label='Elite Schools', color='#1f77b4')
    ax.bar(x + width/2, [other_counts.get(r, 0) for r in regions], width, 
           label='Regional Schools', color='#ff7f0e')
    
    ax.set_xticks(x)
    ax.set_xticklabels([r.title() for r in regions])
    ax.set_ylabel('Number of Admits')
    ax.set_title('Admits by Home Region: Elite vs Regional Schools', fontweight='bold')
    ax.legend()
    
    # -------------------------------------------------------------------------
    # PANEL [1,1]: Error distribution (histogram)
    # -------------------------------------------------------------------------
    # Shows whether errors are symmetric around zero (unbiased model).
    # A good model should have errors centered near zero.
    ax = axes[1, 1]
    
    # Compute errors for all school-tier combinations
    errors = []
    for school in SCHOOLS.keys():
        for tier in ['Rank', 1, 2, 3, 4]:
            if tier in cutoffs.get(school, {}) and tier in CUTOFFS_2024_CALIBRATION.get(school, {}):
                err = cutoffs[school][tier] - CUTOFFS_2024_CALIBRATION[school][tier]
                errors.append(err)
    
    ax.hist(errors, bins=30, edgecolor='black', alpha=0.7)
    ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero error')
    ax.axvline(np.mean(errors), color='green', linestyle='-', linewidth=2, 
               label=f'Mean: {np.mean(errors):.1f}')
    ax.set_xlabel('Error (Simulated - Actual)')
    ax.set_ylabel('Count')
    ax.set_title('Distribution of Cutoff Prediction Errors', fontweight='bold')
    ax.legend()
    
    plt.tight_layout()
    return fig


# Generate and display the results visualization
fig = plot_simulation_results(metrics, matched_students)
plt.show()

## 7. Summary and Conclusions

### 7.1 Key Findings from MLE Analysis

1. **Selection bias is substantial and asymmetric across tiers.** For Tier 1 at elite schools, the observed mean exceeds the true population mean by 150-300 points. For Tier 4, the bias is only 10-30 points because truncation occurs closer to the population mean (the cutoff is not far above $\mu$).

2. **Elite schools exhibit extremely tight T4 distributions.** Northside has $\hat{\sigma} < 5$ for T4, implying virtually all admitted T4 students score 890+. This reflects both extreme competition and ceiling effects near the 900-point maximum.

3. **Regional schools have fundamentally different applicant pools.** Schools like South Shore and King have $\hat{\sigma} > 60$ for T4, indicating heterogeneous applicant pools. This bifurcation suggests modeling elite and regional schools separately may improve accuracy.

### 7.2 Simulation Performance

The v13 model, with parameters tuned via 500 trials of Optuna Bayesian optimization, achieves:

| Metric | Value | Target |
|--------|-------|--------|
| Overall MAE | ~23 points | - |
| Max error | < 85 points | < 80 |
| Max school MAE | ~30 points | < 25 |

**Interpretation**: The model predicts cutoffs within 23 points on average, which is reasonable given the inherent stochasticity of the admissions process. However, some schools (particularly regional schools competing for overlapping applicant pools) remain challenging to predict.

### 7.3 Limitations and Future Work

1. **Unknown applicant counts**: I estimate tier-specific applicant numbers; actual counts are not published by CPS. More accurate applicant data would improve the acceptance rate constraint.

2. **Normality assumption in MLE**: Score distributions may deviate from normality, especially for regional schools with potential bimodality (e.g., students from Academic Centers vs. general population).

3. **Static preferences**: The model assumes preferences are fixed. In reality, students may update preferences strategically based on perceived admission chances.

4. **No sibling/legacy effects**: The model does not account for sibling preferences or Academic Center feeder patterns, which affect demand at specific schools.

5. **"Whack-a-mole" problem**: Regional schools (King, Brooks, Hancock, Lindblom) compete for overlapping applicant pools. Adjusting parameters to improve one school often worsens another. A future v14 model might separate elite and regional schools into distinct sub-models.

In [ ]:
# ==============================================================================
# FINAL SUMMARY
# ==============================================================================
# This cell prints a comprehensive summary of both the MLE analysis and simulation
# performance, including detailed breakdowns by school type and tier.
# ==============================================================================

def mae_rating(mae):
    """Convert MAE to a qualitative rating."""
    if mae < 20:
        return "EXCELLENT"
    elif mae < 30:
        return "GOOD"
    elif mae < 40:
        return "FAIR"
    else:
        return "POOR"

print("=" * 70)
print("FINAL MODEL SUMMARY")
print("=" * 70)
print()

# MLE Analysis Summary
print("MLE ANALYSIS (Section 4-5):")
print(f"  Schools analyzed:        {len(ALL_MLE_RESULTS)}")
print(f"  Tier-level estimates:    {sum(len(r) for r in ALL_MLE_RESULTS.values())}")
print(f"  Data source:             CPS 2024-2025 Official Release (3/14/2025)")
print()

# Key MLE insights
elite_t4_sigmas = [ALL_MLE_RESULTS[s][4].sigma for s in ['Walter Payton', 'Northside', 'Whitney Young', 'Jones', 'Lane Tech']]
regional_t4_sigmas = [ALL_MLE_RESULTS[s][4].sigma for s in ['King', 'South Shore', 'Westinghouse', 'Brooks', 'Hancock', 'Lindblom']]

print("  KEY INSIGHT - T4 sigma bifurcation:")
print(f"    Elite schools avg sigma:    {np.mean(elite_t4_sigmas):.1f}")
print(f"    Regional schools avg sigma: {np.mean(regional_t4_sigmas):.1f}")
print()

# -----------------------------------------------------------------------------
# SIMULATION SUMMARY
# -----------------------------------------------------------------------------
print("SIMULATION (v13, Optuna-tuned, Section 6):")
print(f"  Overall MAE:             {metrics['overall_mae']:.1f} points  [{mae_rating(metrics['overall_mae'])}]")
print(f"  Max Error:               {metrics['max_error']:.0f} points")
print()

# -----------------------------------------------------------------------------
# DETAILED MAE BY SCHOOL TYPE
# -----------------------------------------------------------------------------
print("MAE BY SCHOOL TYPE:")
print("-" * 50)

# Define school groups
regional_schools_list = ['King', 'South Shore', 'Westinghouse', 'Brooks', 'Hancock', 'Lindblom']

# Compute MAE for each group
elite_maes = [metrics['school_errors'][s]['mae'] for s in ELITE_SCHOOLS if s in metrics['school_errors']]
regional_maes = [metrics['school_errors'][s]['mae'] for s in regional_schools_list if s in metrics['school_errors']]

elite_avg = np.mean(elite_maes)
regional_avg = np.mean(regional_maes)

print(f"  Elite Schools:           {elite_avg:.1f} +/- {np.std(elite_maes):.1f}  [{mae_rating(elite_avg)}]")
for school in ELITE_SCHOOLS:
    if school in metrics['school_errors']:
        school_mae = metrics['school_errors'][school]['mae']
        print(f"    {school:<20} {school_mae:>5.1f}  [{mae_rating(school_mae)}]")

print()
print(f"  Regional Schools:        {regional_avg:.1f} +/- {np.std(regional_maes):.1f}  [{mae_rating(regional_avg)}]")
for school in regional_schools_list:
    if school in metrics['school_errors']:
        school_mae = metrics['school_errors'][school]['mae']
        print(f"    {school:<20} {school_mae:>5.1f}  [{mae_rating(school_mae)}]")

print()

# -----------------------------------------------------------------------------
# DETAILED MAE BY TIER
# -----------------------------------------------------------------------------
print("MAE BY TIER:")
print("-" * 50)

# Compute cutoffs from matched students for tier-level analysis
cutoffs = compute_cutoffs(matched_students)

# Calculate errors by tier
tier_errors = {tier: [] for tier in ['Rank', 1, 2, 3, 4]}

for school in SCHOOLS.keys():
    for tier in ['Rank', 1, 2, 3, 4]:
        if tier in cutoffs.get(school, {}) and tier in CUTOFFS_2024_CALIBRATION.get(school, {}):
            err = abs(cutoffs[school][tier] - CUTOFFS_2024_CALIBRATION[school][tier])
            tier_errors[tier].append(err)

for tier in ['Rank', 1, 2, 3, 4]:
    if tier_errors[tier]:
        tier_label = f"Tier {tier}" if tier != 'Rank' else "Rank"
        mean_err = np.mean(tier_errors[tier])
        std_err = np.std(tier_errors[tier])
        print(f"  {tier_label:<10} MAE: {mean_err:>5.1f} +/- {std_err:>5.1f}  [{mae_rating(mean_err)}]  (n={len(tier_errors[tier])})")

print()

# -----------------------------------------------------------------------------
# MAE BY TIER AND SCHOOL TYPE (CROSS-TABULATION)
# -----------------------------------------------------------------------------
print("MAE BY TIER AND SCHOOL TYPE:")
print("-" * 50)

# Calculate errors by tier for each school type
elite_tier_errors = {tier: [] for tier in ['Rank', 1, 2, 3, 4]}
regional_tier_errors = {tier: [] for tier in ['Rank', 1, 2, 3, 4]}

for school in SCHOOLS.keys():
    is_elite = school in ELITE_SCHOOLS
    target_dict = elite_tier_errors if is_elite else regional_tier_errors
    
    for tier in ['Rank', 1, 2, 3, 4]:
        if tier in cutoffs.get(school, {}) and tier in CUTOFFS_2024_CALIBRATION.get(school, {}):
            err = abs(cutoffs[school][tier] - CUTOFFS_2024_CALIBRATION[school][tier])
            target_dict[tier].append(err)

# Print cross-tabulation
print(f"  {'Tier':<10} {'Elite MAE':>15} {'Regional MAE':>18}")
print(f"  {'-'*10} {'-'*15} {'-'*18}")

for tier in ['Rank', 1, 2, 3, 4]:
    tier_label = f"Tier {tier}" if tier != 'Rank' else "Rank"
    elite_mean = np.mean(elite_tier_errors[tier]) if elite_tier_errors[tier] else float('nan')
    regional_mean = np.mean(regional_tier_errors[tier]) if regional_tier_errors[tier] else float('nan')
    elite_rating = mae_rating(elite_mean) if not np.isnan(elite_mean) else "N/A"
    regional_rating = mae_rating(regional_mean) if not np.isnan(regional_mean) else "N/A"
    print(f"  {tier_label:<10} {elite_mean:>6.1f} [{elite_rating:<9}] {regional_mean:>6.1f} [{regional_rating:<9}]")

print()

# -----------------------------------------------------------------------------
# VALIDATION SUMMARY
# -----------------------------------------------------------------------------
print("VALIDATION (5-seed stability test):")
print(f"  MAE range:               [{val_df['mae'].min():.1f}, {val_df['mae'].max():.1f}]")
print(f"  MAE mean +/- std:        {val_df['mae'].mean():.1f} +/- {val_df['mae'].std():.1f}")
print()

# -----------------------------------------------------------------------------
# TARGET CHECK
# -----------------------------------------------------------------------------
max_school_mae = max(metrics['school_errors'][s]['mae'] for s in SCHOOLS if s in metrics['school_errors'])

print("ROBUSTNESS CHECK:")
print("-" * 50)
print(f"  Overall MAE:             {metrics['overall_mae']:.1f} points  [{mae_rating(metrics['overall_mae'])}]")
print(f"  Elite Schools MAE:       {elite_avg:.1f} points  [{mae_rating(elite_avg)}]")
print(f"  Regional Schools MAE:    {regional_avg:.1f} points  [{mae_rating(regional_avg)}]")
print(f"  Max Single School MAE:   {max_school_mae:.1f} points  [{mae_rating(max_school_mae)}]")
print(f"  Max Single Error:        {metrics['max_error']:.0f} points")
print()

# Tier-level ratings
print("  Tier-level performance:")
tier_ratings = {'EXCELLENT': 0, 'GOOD': 0, 'FAIR': 0, 'POOR': 0}
for tier in [1, 2, 3, 4]:
    tier_mae = np.mean(tier_errors[tier]) if tier_errors[tier] else float('nan')
    rating = mae_rating(tier_mae)
    tier_ratings[rating] += 1
    print(f"    Tier {tier}:               {tier_mae:>5.1f}  [{rating}]")

print()

# School-type ratings
print("  School-type performance:")
print(f"    Elite Schools:         {elite_avg:>5.1f}  [{mae_rating(elite_avg)}]")
print(f"    Regional Schools:      {regional_avg:>5.1f}  [{mae_rating(regional_avg)}]")

print()
print("=" * 70)

# -----------------------------------------------------------------------------
# INTERPRETATION
# -----------------------------------------------------------------------------
print()
print("INTERPRETATION:")
print("-" * 70)
print()
print("MAE Rating Scale:")
print("  EXCELLENT (< 20):  Predictions within ~2% of 900-point scale")
print("                     High confidence for admission decisions")
print("                     Student can reliably target schools at this level")
print()
print("  GOOD (20-30):      Predictions within ~2-3% of scale")
print("                     Solid guidance for most students")
print("                     Add 20-point safety margin to predictions")
print()
print("  FAIR (30-40):      Predictions within ~3-4% of scale")
print("                     Useful for general guidance, not precise targeting")
print("                     Add 30-point safety margin; consider backup schools")
print()
print("  POOR (40+):        Predictions have >4% uncertainty")
print("                     Use only for rough estimates")
print("                     Wide confidence intervals needed")
print()

# Count ratings
print("Model Performance Summary:")
excellent_count = sum(1 for s in metrics['school_errors'].values() if s['mae'] < 20)
good_count = sum(1 for s in metrics['school_errors'].values() if 20 <= s['mae'] < 30)
fair_count = sum(1 for s in metrics['school_errors'].values() if 30 <= s['mae'] < 40)
poor_count = sum(1 for s in metrics['school_errors'].values() if s['mae'] >= 40)

print(f"  Schools by rating:  {excellent_count} EXCELLENT, {good_count} GOOD, {fair_count} FAIR, {poor_count} POOR")
print(f"  Tiers by rating:    {tier_ratings['EXCELLENT']} EXCELLENT, {tier_ratings['GOOD']} GOOD, {tier_ratings['FAIR']} FAIR, {tier_ratings['POOR']} POOR")
print()

# Specific interpretations based on results
print("Practical Implications:")
print()

if elite_avg < regional_avg:
    print(f"  Elite Schools ({mae_rating(elite_avg)}, MAE {elite_avg:.1f}):")
    print("    Students targeting Payton, Northside, Lane Tech, etc. can rely on")
    print("    predictions with reasonable confidence. The model captures the")
    print("    predictable behavior of high-scoring North/Loop applicants.")
    print()
    print(f"  Regional Schools ({mae_rating(regional_avg)}, MAE {regional_avg:.1f}):")
    print("    Predictions for King, Brooks, South Shore, etc. have more uncertainty.")
    print("    These schools compete for overlapping West/South applicant pools,")
    print("    and student preferences are more variable. Use wider margins.")
else:
    print(f"  Regional Schools ({mae_rating(regional_avg)}, MAE {regional_avg:.1f}):")
    print("    Predictions for regional schools are as reliable as elite schools.")
    print()
    print(f"  Elite Schools ({mae_rating(elite_avg)}, MAE {elite_avg:.1f}):")
    print("    Elite school predictions may have more uncertainty due to")
    print("    extreme competition at the 900-point ceiling.")

print()

# Identify best and worst predicted tiers
tier_mae_list = [(tier, np.mean(tier_errors[tier])) for tier in [1, 2, 3, 4] if tier_errors[tier]]
best_tier = min(tier_mae_list, key=lambda x: x[1])
worst_tier = max(tier_mae_list, key=lambda x: x[1])

print("Tier-Level Guidance:")
print(f"  Best predicted:  Tier {best_tier[0]} ({mae_rating(best_tier[1])}, MAE {best_tier[1]:.1f})")
print(f"  Worst predicted: Tier {worst_tier[0]} ({mae_rating(worst_tier[1])}, MAE {worst_tier[1]:.1f})")
print()

if worst_tier[0] == 4:
    print("  Tier 4 (affluent areas) is hardest to predict because high-scoring")
    print("  students have many options and may choose private schools or suburbs.")
elif worst_tier[0] == 1:
    print("  Tier 1 (disadvantaged areas) is hardest to predict due to wider")
    print("  score distributions and more variable application behavior.")
elif worst_tier[0] in [2, 3]:
    print(f"  Tier {worst_tier[0]} shows moderate prediction difficulty, likely due to")
    print("  students in this tier having diverse school preferences.")

print()

# Overall assessment
overall_rating = mae_rating(metrics['overall_mae'])
print("Overall Assessment:")
print(f"  Model Rating: {overall_rating} (Overall MAE: {metrics['overall_mae']:.1f})")
print()

if overall_rating == "EXCELLENT":
    print("  This model provides highly reliable predictions across all schools")
    print("  and tiers. Students can use these predictions with high confidence")
    print("  for admission planning.")
elif overall_rating == "GOOD":
    print("  This model provides solid predictions suitable for most students.")
    print("  Recommended use:")
    print("    - Students 30+ points above predicted cutoff: Likely admission")
    print("    - Students within 30 points: Competitive, include backup options")
    print("    - Students 30+ points below: Consider this a reach school")
elif overall_rating == "FAIR":
    print("  This model provides useful guidance but with notable uncertainty.")
    print("  Recommended use:")
    print("    - Use predictions as rough estimates, not precise targets")
    print("    - Always include 2-3 backup schools at lower cutoff levels")
    print("    - Consider year-to-year variation in actual cutoffs")
else:
    print("  This model has significant prediction uncertainty.")
    print("  Use predictions only for general orientation, not decision-making.")

print()
print("=" * 70)

# Monte Carlo Admission Probability Simulator

This tool estimates your probability of admission to **each** of the 11 Selective Enrollment High Schools based on your score and tier.

## How It Works

1. **Enter your profile**: Composite score (400-900) and tier (1-4)
2. **Simulation**: For each school, we run 100 simulations where that school is your #1 choice
3. **Competition**: Each simulation includes ~2,000 students with realistic score distributions competing for scaled seat counts

The matching algorithm replicates the actual CPS process:
- **Phase 1 (30% of seats)**: Rank-based matching - highest scores get first pick regardless of tier
- **Phase 2 (70% of seats)**: Tier-based matching - seats allocated within each tier by score

## Output

For each school, you'll see:
- **Probability**: Percentage of simulations where you were admitted (if that school was your #1 choice)
- **Assessment**: Qualitative rating based on probability

| Probability | Assessment |
|-------------|------------|
| 95%+ | VIRTUALLY CERTAIN |
| 90-94% | EXTREMELY LIKELY |
| 85-89% | VERY LIKELY |
| 80-84% | HIGHLY LIKELY |
| 75-79% | LIKELY |
| 70-74% | PROBABLE |
| 65-69% | GOOD CHANCE |
| 60-64% | FAVORABLE |
| 55-59% | SLIGHT EDGE |
| 50-54% | TOSS-UP |
| 40-49% | COMPETITIVE |
| 30-39% | REACH |
| 20-29% | STRETCH |
| 10-19% | LONG SHOT |
| 1-9% | UNLIKELY |
| 0% | NO CHANCE |

## Summary Categories

At the end, schools are grouped into:
- **Very Likely (80%+)**: Strong safety schools
- **Likely (55-79%)**: Good options with reasonable confidence  
- **Competitive (30-54%)**: Match schools - could go either way
- **Reach (10-29%)**: Ambitious choices - possible but not probable

In [ ]:
# ============================================================================
# MONTE CARLO ADMISSION PROBABILITY SIMULATOR
# ============================================================================
# This tool estimates your probability of admission to each SEHS school
# by running simulations where each school is ranked as your #1 choice.
#
# For each school, we run 50 simulations to estimate:
#   - Probability of admission if that school is your top choice
#   - Overall admission landscape for your score/tier profile
# ============================================================================

import numpy as np
import pandas as pd
from IPython.display import display
from sehs_simulation_v13 import generate_students, calculate_preferences, SCHOOLS
from sehs_data import ADMISSIONS

# ----------------------------------------------------------------------------
# SCHOOL LIST (ordered by prestige/competitiveness)
# ----------------------------------------------------------------------------
SCHOOL_LIST = [
    (1, "Walter Payton", 99),
    (2, "Northside", 98),
    (3, "Whitney Young", 96),
    (4, "Jones", 93),
    (5, "Lane Tech", 92),
    (6, "Lindblom", 81),
    (7, "Hancock", 77),
    (8, "King", 77),
    (9, "Brooks", 76),
    (10, "Westinghouse", 71),
    (11, "South Shore", 66),
]

SCHOOL_NAMES = [s[1] for s in SCHOOL_LIST]


def run_match_with_user_prefs(student_df: pd.DataFrame, user_idx: int, user_prefs: list, scale_factor: float = 1.0) -> pd.DataFrame:
    """
    Custom matching function that preserves user's preference list.
    
    Args:
        student_df: DataFrame of students
        user_idx: Index of the user in the DataFrame
        user_prefs: User's preference list (preserved, not recalculated)
        scale_factor: Seat scaling factor (n_students / 22000)
    """
    # Initialize seat counts - SCALED to match student population size
    seats = {}
    for name, school in SCHOOLS.items():
        # Scale seats proportionally to maintain realistic competition
        scaled_seats = max(int(school.seats * scale_factor), 1)
        seats[name] = {
            'Rank': max(int(scaled_seats * ADMISSIONS['rank_fraction']), 1),
            1: max(int(scaled_seats * ADMISSIONS['tier_fraction']), 1),
            2: max(int(scaled_seats * ADMISSIONS['tier_fraction']), 1),
            3: max(int(scaled_seats * ADMISSIONS['tier_fraction']), 1),
            4: max(int(scaled_seats * ADMISSIONS['tier_fraction']), 1),
        }
    
    student_df = student_df.copy()
    
    # Initialize Preferences column with None (must exist before setting values)
    student_df['Preferences'] = None
    student_df['Matched'] = None
    student_df['MatchType'] = None
    
    # Calculate preferences for all students EXCEPT the user
    for idx in student_df.index:
        if idx != user_idx:
            prefs = calculate_preferences(student_df.loc[idx])
            student_df.at[idx, 'Preferences'] = prefs
        else:
            # Use object dtype assignment for list
            student_df.at[idx, 'Preferences'] = user_prefs
    
    # Phase 1: Rank-based (top 30%)
    rank_sorted = student_df.sort_values(['Score', 'TieBreaker'], ascending=[False, True])
    
    for idx in rank_sorted.index:
        prefs = rank_sorted.loc[idx, 'Preferences']
        if not isinstance(prefs, list) or len(prefs) == 0:
            continue
        
        for school in prefs:
            if school in seats and seats[school]['Rank'] > 0:
                student_df.loc[idx, 'Matched'] = school
                student_df.loc[idx, 'MatchType'] = 'Rank'
                seats[school]['Rank'] -= 1
                break
    
    # Phase 2: Tier-based (remaining 70%)
    for tier in [1, 2, 3, 4]:
        unmatched = student_df[student_df['Matched'].isna()]
        tier_students = unmatched[unmatched['Tier'] == tier]
        tier_sorted = tier_students.sort_values(['Score', 'TieBreaker'], ascending=[False, True])
        
        for idx in tier_sorted.index:
            if student_df.loc[idx, 'Matched'] is not None:
                continue
            
            prefs = student_df.loc[idx, 'Preferences']
            if not isinstance(prefs, list) or len(prefs) == 0:
                continue
            
            for school in prefs:
                if school in seats and seats[school][tier] > 0:
                    student_df.loc[idx, 'Matched'] = school
                    student_df.loc[idx, 'MatchType'] = 'Tier'
                    seats[school][tier] -= 1
                    break
    
    return student_df


def get_assessment(prob):
    """Convert probability to qualitative assessment with fine-grained ratings."""
    if prob >= 95:
        return "VIRTUALLY CERTAIN"
    elif prob >= 90:
        return "EXTREMELY LIKELY"
    elif prob >= 85:
        return "VERY LIKELY"
    elif prob >= 80:
        return "HIGHLY LIKELY"
    elif prob >= 75:
        return "LIKELY"
    elif prob >= 70:
        return "PROBABLE"
    elif prob >= 65:
        return "GOOD CHANCE"
    elif prob >= 60:
        return "FAVORABLE"
    elif prob >= 55:
        return "SLIGHT EDGE"
    elif prob >= 50:
        return "TOSS-UP"
    elif prob >= 40:
        return "COMPETITIVE"
    elif prob >= 30:
        return "REACH"
    elif prob >= 20:
        return "STRETCH"
    elif prob >= 10:
        return "LONG SHOT"
    elif prob > 0:
        return "UNLIKELY"
    else:
        return "NO CHANCE"


def run_full_probability_analysis(
    user_score: float,
    user_tier: int,
    n_simulations: int = 100,
    n_students: int = 2000
):
    """
    Run Monte Carlo simulation for ALL schools as potential #1 choice.
    
    Args:
        user_score: User's composite score (400-900)
        user_tier: User's tier (1-4)
        n_simulations: Simulations per school (default 50)
        n_students: Students per simulation (default 2000)
    
    Returns:
        DataFrame with admission probabilities for each school
    """
    
    print(f"Your Profile: Score = {user_score}, Tier = {user_tier}")
    print(f"Running {n_simulations} simulations for each of 11 schools...")
    print(f"Total simulations: {n_simulations * 11}")
    print()
    
    results = []
    
    for school_num, top_choice, prestige in SCHOOL_LIST:
        print(f"  Simulating {SCHOOLS[top_choice].short_name}...", end=" ")
        
        # Build preference list: this school first, then others by prestige
        user_prefs = [top_choice]
        for num, name, prest in SCHOOL_LIST:
            if name != top_choice:
                user_prefs.append(name)
        user_prefs = user_prefs[:6]  # Max 6 choices
        
        # Determine user region based on school location
        if top_choice in ["Walter Payton", "Northside", "Lane Tech"]:
            user_region = 'north'
        elif top_choice in ["Whitney Young", "Jones"]:
            user_region = 'loop'
        elif top_choice == "Westinghouse":
            user_region = 'west'
        else:
            user_region = 'south'
        
        admission_count = 0
        
        for sim in range(n_simulations):
            # Generate student population
            students = generate_students(n=n_students, seed=sim * 100 + school_num * 1000)
            
            # Create user row
            user_row = pd.DataFrame([{
                'Tier': user_tier,
                'Region': user_region,
                'Score': user_score,
                'Lat': SCHOOLS[top_choice].lat,
                'Lon': SCHOOLS[top_choice].lon,
                'TieBreaker': np.random.random()
            }])
            
            students = pd.concat([students, user_row], ignore_index=True)
            user_idx = len(students) - 1
            
            # Run matching with scaled seats
            scale_factor = n_students / 22000.0
            matched_students = run_match_with_user_prefs(students, user_idx, user_prefs, scale_factor)
            
            # Check if user got into their top choice
            user_result = matched_students.loc[user_idx, 'Matched']
            if user_result == top_choice:
                admission_count += 1
        
        prob = admission_count / n_simulations * 100
        print(f"{prob:.0f}%")
        
        results.append({
            'Rank': school_num,
            'School': SCHOOLS[top_choice].short_name,
            'Full Name': top_choice,
            'Seats': SCHOOLS[top_choice].seats,
            'Probability': prob,
            'Assessment': get_assessment(prob),
            'Admissions': admission_count,
            'Simulations': n_simulations
        })
    
    results_df = pd.DataFrame(results)
    
    # Display results
    print()
    print("=" * 70)
    print("ADMISSION PROBABILITY RESULTS")
    print(f"Score: {user_score} | Tier: {user_tier} | Simulations per school: {n_simulations}")
    print("=" * 70)
    print()
    print(f"{'#':<3} {'School':<12} {'Seats':>6} {'Probability':>14} {'Assessment':<18}")
    print("-" * 60)
    
    for _, row in results_df.iterrows():
        prob_str = f"{row['Probability']:5.1f}%"
        print(f"{row['Rank']:<3} {row['School']:<12} {row['Seats']:>6} {prob_str:>14} {row['Assessment']:<18}")
    
    print()
    
    # Summary with more granular breakdown
    very_likely = results_df[results_df['Probability'] >= 80]['School'].tolist()
    likely = results_df[(results_df['Probability'] >= 55) & (results_df['Probability'] < 80)]['School'].tolist()
    competitive = results_df[(results_df['Probability'] >= 30) & (results_df['Probability'] < 55)]['School'].tolist()
    reach = results_df[(results_df['Probability'] >= 10) & (results_df['Probability'] < 30)]['School'].tolist()
    
    print("SUMMARY:")
    if very_likely:
        print(f"  Very Likely (>=80%): {', '.join(very_likely)}")
    if likely:
        print(f"  Likely (55-79%): {', '.join(likely)}")
    if competitive:
        print(f"  Competitive (30-54%): {', '.join(competitive)}")
    if reach:
        print(f"  Reach (10-29%): {', '.join(reach)}")
    
    if not very_likely and not likely and not competitive:
        print("  All schools are reaches or long shots for your profile.")
        print("  Consider improving your score or exploring other options.")
    
    return results_df


# ============================================================================
# INTERACTIVE INPUT
# ============================================================================
print("=" * 50)
print("SEHS ADMISSION PROBABILITY CALCULATOR")
print("=" * 50)
print()
print("This tool calculates your admission probability")
print("for each school if you rank it as #1.")
print()

try:
    user_score = float(input("Enter your composite score (400-900): "))
    user_score = max(400, min(900, user_score))
    
    user_tier = int(input("Enter your tier (1-4): "))
    user_tier = max(1, min(4, user_tier))
    
    print()
    
    # Run full analysis
    results = run_full_probability_analysis(
        user_score=user_score,
        user_tier=user_tier,
        n_simulations=100,
        n_students=2000
    )
    
except ValueError as e:
    print(f"Invalid input: {e}")
    print("Please enter numeric values.")
except KeyboardInterrupt:
    print("\nSimulation cancelled.")


## Statistical Framework for the Monte Carlo Admission Simulator

### Problem Formulation

We seek to estimate the probability $P(A_s | X = x, T = t)$ that a student with composite score $x$ and tier $t$ gains admission to school $s$, conditional on ranking school $s$ as their first choice.

Since the admission process involves stochastic competition from other applicants, we employ Monte Carlo simulation to estimate these probabilities empirically.

---

### Random Variables and Sample Space

Let the following random variables be defined:

| Variable | Description | Support |
|----------|-------------|---------|
| $X_i$ | Composite score of student $i$ | $[400, 900] \subset \mathbb{R}$ |
| $T_i$ | Tier assignment of student $i$ | $\{1, 2, 3, 4\}$ |
| $R_i$ | Geographic region of student $i$ | $\{\text{North}, \text{Loop}, \text{West}, \text{South}\}$ |
| $\mathbf{P}_i$ | Preference list of student $i$ | $\mathcal{S}^{\leq 6}$ (ordered subsets of schools) |
| $M_i$ | Matched school for student $i$ | $\mathcal{S} \cup \{\emptyset\}$ |

where $\mathcal{S} = \{s_1, s_2, \ldots, s_{11}\}$ denotes the set of 11 SEHS schools, and $\emptyset$ represents the unmatched state.

The **sample space** for a single simulation is:

$$\Omega = \prod_{i=1}^{n} \left( [400, 900] \times \{1,2,3,4\} \times \{N, L, W, S\} \times \mathcal{S}^{\leq 6} \right)$$

where $n \approx 2000$ is the simulated applicant pool size.

---

### Score Distribution Model

Scores are generated from a **skew-normal distribution** conditional on region and tier:

$$X_i \mid (R_i = r, T_i = t) \sim \text{SkewNormal}(\xi_{r,t}, \omega_{r,t}, \alpha_{r,t})$$

The subscript notation $(r, t)$ indexes a specific combination of region $r \in \{\text{North}, \text{Loop}, \text{West}, \text{South}\}$ and tier $t \in \{1, 2, 3, 4\}$. This yields a $4 \times 4 = 16$ parameter grid, reflecting the empirical observation that score distributions vary systematically across both geographic and socioeconomic dimensions. For example, $\xi_{\text{North}, 4}$ denotes the location parameter for Tier 4 students residing in the North region—a subpopulation that tends to exhibit higher scores due to correlation between neighborhood characteristics and academic preparation.

The parameters are:
- $\xi_{r,t}$ (**location**): Controls the central tendency of the distribution. Higher values shift the distribution rightward toward higher scores.
- $\omega_{r,t}$ (**scale**): Controls the dispersion. Larger values produce wider score spreads within the subpopulation.
- $\alpha_{r,t}$ (**shape/skewness**): Controls asymmetry. Positive values ($\alpha > 0$) produce right-skewed distributions (long tail toward high scores), while negative values ($\alpha < 0$) produce left-skewed distributions (mass concentrated at high scores with a tail toward lower scores).

The probability density function is:

$$f(x; \xi, \omega, \alpha) = \frac{2}{\omega} \phi\left(\frac{x - \xi}{\omega}\right) \Phi\left(\alpha \cdot \frac{x - \xi}{\omega}\right)$$

where $\phi(\cdot)$ and $\Phi(\cdot)$ are the standard normal PDF and CDF, respectively.

**Justification for the skew-normal family:** Empirical score distributions in standardized testing typically exhibit asymmetry—particularly ceiling effects near the maximum score (900) for high-performing subpopulations. The skew-normal distribution captures this behavior parsimoniously with a single additional parameter beyond the normal distribution, while remaining analytically tractable.

Scores are subsequently clipped to enforce the domain constraint:

$$\tilde{X}_i = \min(\max(X_i, 400), 900)$$

---

### The Matching Mechanism

The CPS admission process implements a **serial dictatorship** mechanism, a classical assignment algorithm from mechanism design theory. This procedure is **strategy-proof**, meaning students have no incentive to misrepresent their true preferences—truthful reporting is a dominant strategy.

Define the capacity constraints:

- $C_s^{\text{Rank}}$: Rank-based seats at school $s$ (30% of total capacity)
- $C_s^{(t)}$: Tier-$t$ seats at school $s$ (17.5% of total capacity, for $t \in \{1,2,3,4\}$)

The algorithm proceeds in two phases:

**Phase 1 (Rank-Based Matching):** 

All students, regardless of tier, are sorted by composite score in descending order. Beginning with the highest-scoring student, each student is considered in sequence. When a student is processed, the algorithm examines their preference list from most-preferred to least-preferred school. The student is assigned to the first school on their list that still has available rank-based seats. Once assigned, that school's rank seat count is decremented by one. If no preferred school has available rank seats, the student remains unmatched and proceeds to Phase 2.

This phase allocates 30% of each school's capacity to the highest-scoring applicants citywide, irrespective of their tier classification.

**Phase 2 (Tier-Based Matching):**

The remaining 70% of seats are allocated within tier. For each tier $t$ from 1 to 4, the algorithm considers only unmatched students belonging to that tier, sorted by score in descending order. Each student is assigned to their highest-preference school that still has available tier-$t$ seats. This ensures that students compete primarily against others in their same socioeconomic tier for the majority of seats.

#### Formalization via Permutations

To express this algorithm precisely, we introduce the concept of a **permutation**. In group theory, a permutation of a set $S$ is a bijective function $\pi: S \to S$. The set of all permutations of $n$ elements forms the **symmetric group** $S_n$, which has $n!$ elements. Permutations are useful here because the matching algorithm processes students in a specific order determined by their scores.

**Notation:** We write $\pi(k) = i$ to mean "the student in position $k$ of the ordering is student $i$." Equivalently, $\pi^{-1}(i) = k$ gives student $i$'s position in the ordering.

Let $\pi: \{1, \ldots, n\} \to \{1, \ldots, n\}$ be the permutation that orders students by score in descending order. Formally:

$$X_{\pi(1)} \geq X_{\pi(2)} \geq \cdots \geq X_{\pi(n)}$$

**Tie-breaking:** When two students $i$ and $j$ have identical scores ($X_i = X_j$), the ordering is ambiguous. To resolve this, each student is assigned a **tie-breaker** $U_i \sim \text{Uniform}(0, 1)$ independently at random. The permutation $\pi$ then satisfies:

$$\pi^{-1}(i) < \pi^{-1}(j) \iff (X_i > X_j) \text{ or } (X_i = X_j \text{ and } U_i < U_j)$$

This ensures a strict total ordering exists with probability 1.

The matching function $M: \{1, \ldots, n\} \to \mathcal{S} \cup \{\emptyset\}$ is then computed as:

$$M_i = \begin{cases}
s^* & \text{if } \exists s^* \in \mathbf{P}_i : C_{s^*}^{(\cdot)} > 0 \text{ when student } i \text{ is processed} \\
\emptyset & \text{otherwise}
\end{cases}$$

where $C_{s^*}^{(\cdot)}$ denotes the relevant capacity (rank or tier-specific) at the time of processing.

---

### Monte Carlo Estimator

For each school $s$, we run $B = 100$ independent simulations. Let $Y_s^{(b)} \in \{0, 1\}$ be the indicator that the user is admitted to school $s$ in simulation $b$:

$$Y_s^{(b)} = \mathbf{1}\{M_{\text{user}}^{(b)} = s\}$$

The Monte Carlo estimator for $P(A_s | X = x, T = t)$ is:

$$\hat{p}_s = \frac{1}{B} \sum_{b=1}^{B} Y_s^{(b)}$$

By the Strong Law of Large Numbers:

$$\hat{p}_s \xrightarrow{a.s.} P(A_s | X = x, T = t) \quad \text{as } B \to \infty$$

#### Standard Error and Confidence Intervals

Since $Y_s^{(b)} \sim \text{Bernoulli}(p_s)$ and the simulations are independent, the variance of the estimator is:

$$\text{Var}(\hat{p}_s) = \frac{p_s(1 - p_s)}{B}$$

The standard error, estimated from the data, is:

$$\widehat{\text{SE}}(\hat{p}_s) = \sqrt{\frac{\hat{p}_s(1 - \hat{p}_s)}{B}}$$

For $B = 100$ simulations:
- If $\hat{p}_s = 0.50$, then $\widehat{\text{SE}} = \sqrt{0.25/100} = 0.05$, yielding a 95% CI of approximately $\pm 10\%$
- If $\hat{p}_s = 0.80$, then $\widehat{\text{SE}} = \sqrt{0.16/100} = 0.04$, yielding a 95% CI of approximately $\pm 8\%$

Compared to $B = 50$ simulations (where the 95% CI would be $\pm 14\%$ at $\hat{p} = 0.50$), doubling the simulation count reduces the confidence interval width by a factor of $\sqrt{2} \approx 1.41$. This reflects the general principle that Monte Carlo standard errors decrease at rate $O(1/\sqrt{B})$.

---

### Assumptions and Limitations

1. **Independence across simulations**: Each simulation $b$ draws an independent applicant pool from the score distribution model. This assumes the score-generating process is stationary across hypothetical replications.

2. **Deterministic preferences for competitors**: Other students' preferences are computed via a deterministic utility function that depends on school prestige, geographic distance, and region-specific penalties. Only the user's preferences are fixed exogenously (set to the school being tested as #1 choice). 

   *In plain terms: your simulated competitors always "decide" where to apply using the same formula based on where they live and their scores. You, however, get to specify your own preference list. This means the simulation captures realistic competition patterns while letting you explore "what if I ranked school X first?" and running 100 simulations based of that set of rotating assumptions*

3. **Proportional scaling**: Seat counts are scaled by $\lambda = n/22000$ to maintain realistic competition ratios:
   $$C_s^{(\cdot)} \leftarrow \lfloor \lambda \cdot C_s^{(\cdot)} \rfloor$$
   This preserves the seats-to-applicants ratio when using a smaller simulated population.

4. **Model misspecification**: The skew-normal score distributions are calibrated via Optuna hyperparameter optimization (a Bayesian optimization framework using Tree-structured Parzen Estimators) to minimize discrepancy between simulated and historical cutoff scores. However, this calibration targets aggregate statistics (cutoffs, acceptance rates) rather than the full distributional shape. Consequently, the fitted parameters $\{\xi_{r,t}, \omega_{r,t}, \alpha_{r,t}\}$ may not perfectly capture features such as:
   - Multimodality in the true score distribution
   - Heavy tails or outlier behavior
   - Temporal non-stationarity (year-to-year variation in applicant pools)
   - Correlation structure between scores and unobserved confounders
   
   The simulation should therefore be interpreted as providing *plausible* probability estimates under the assumed model, not exact forecasts.

5. **No strategic behavior**: Students are assumed to submit preferences truthfully. This assumption is justified by the strategy-proofness of serial dictatorship mechanisms—no student can improve their outcome by misrepresenting preferences, regardless of others' strategies.

---

### Interpretation

The output $\hat{p}_s$ represents the **frequentist probability** of admission under repeated sampling from the model. It answers:

> "If we drew many applicant pools from this model and ran the CPS matching algorithm each time, in what fraction would a student with your profile be admitted to school $s$?"

This differs from a Bayesian posterior probability, which would incorporate prior beliefs about model parameters. The frequentist interpretation treats the model parameters as fixed (though estimated) and quantifies uncertainty arising solely from the randomness in competitor draws.

# Animated Visualization: The Serial Dictatorship Match Process

This animation provides a visual demonstration of how the CPS selective enrollment matching algorithm processes students. Each student is represented as a dot, with:

- **Color**: Indicates the student's home region (North=blue, Loop=orange, West=green, South=red)
- **Shade intensity**: Indicates tier (darker = higher tier)

The animation shows:
1. Students sorted by score (rank phase) or by tier+score (tier phase)
2. Each student "moving" from the applicant pool to their matched school
3. Real-time seat availability updates
4. Unmatched students collecting in the "Unmatched" area

This visualization helps illustrate the competitive dynamics and geographic patterns in SEHS admissions.

In [ ]:
# ============================================================================
# ANIMATED MATCH VISUALIZATION
# ============================================================================
# This cell creates an animated visualization of the serial dictatorship
# matching process, showing how students flow from the applicant pool to 
# their matched schools.
#
# Features:
#   - Log-scaled Y-axis to spread out top scores for visibility
#   - Students "pop" from top of stack and move to matched school
#   - Cap indicators show when tier/rank seats are exhausted
#   - Complete simulation: Phase 1 (Rank 30%) + Phase 2 (Tiers 1-4, 70%)
#   - Rank matches shown in black, tier matches in region colors
# ============================================================================

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as mpatches
import numpy as np
from IPython.display import HTML
from collections import defaultdict

# Import simulation functions from v13 module
from sehs_simulation_v13 import generate_students, calculate_preferences, SCHOOLS

# ----------------------------------------------------------------------------
# REGION COLOR SCHEME
# Region determines hue, tier determines intensity (darker = higher tier)
# Rank-based matches are shown in black
# ----------------------------------------------------------------------------
REGION_BASE_COLORS = {
    'north': '#3182bd',   # Blue
    'loop':  '#fd8d3c',   # Orange  
    'west':  '#31a354',   # Green
    'south': '#e6550d',   # Red-orange
}

TIER_COLORS = {
    1: '#fee5d9',  # Light (T1)
    2: '#fcae91',  # Medium-light (T2)
    3: '#fb6a4a',  # Medium-dark (T3)
    4: '#cb181d',  # Dark (T4)
}

def get_student_color(region, tier, match_type='tier'):
    """
    Generate color for a student based on region, tier, and match type.
    
    Args:
        region: Student's home region
        tier: Student's tier (1-4)
        match_type: 'rank' (black) or 'tier' (region-colored)
    
    Returns:
        RGBA tuple
    """
    import matplotlib.colors as mcolors
    
    if match_type == 'rank':
        # Rank matches are black
        return (0.1, 0.1, 0.1, 0.9)
    
    base = REGION_BASE_COLORS.get(region, '#888888')
    rgb = mcolors.to_rgb(base)
    
    # Tier-based intensity: T1=lighter, T4=darker
    tier_alphas = {1: 0.5, 2: 0.65, 3: 0.8, 4: 1.0}
    alpha = tier_alphas.get(tier, 0.7)
    
    return (*rgb, alpha)


def score_to_y(score, y_min=0, y_max=10):
    """
    Convert score to Y position using exponential-like scaling.
    Bunches low scores at bottom, spreads out high scores at top.
    
    Uses a power transform with exponent > 1: higher scores get more vertical space.
    This highlights the fewer high-scoring students at the top.
    """
    # Normalize score to 0-1 range (assuming 400-900 score range)
    normalized = (score - 400) / 500
    normalized = np.clip(normalized, 0, 1)
    
    # Apply power transform to spread out TOP scores (power > 1)
    # Power of 2.5 means: bottom 50% of scores use ~18% of vertical space
    #                     top 20% of scores use ~45% of vertical space
    transformed = np.power(normalized, 2.5)
    
    # Map to y range
    return y_min + transformed * (y_max - y_min)


def create_match_animation(
    students_df, 
    sample_size=400, 
    interval_ms=40,
    students_per_frame=3,
    seed=42
):
    """
    Create an animated visualization of the SEHS matching process.
    
    The animation shows the complete matching algorithm:
    - Phase 1: Rank-based (30% of seats, highest scorers first)
    - Phase 2: Tier-based (70% of seats, by tier then by score)
    
    Visual features:
    - Students sorted by score, popped from top of stack
    - Log-scaled Y-axis spreads out top scores
    - Rank matches in black, tier matches in region colors
    - Cap indicators when seats exhausted
    """
    
    np.random.seed(seed)
    
    # -------------------------------------------------------------------------
    # SAMPLE AND PREPARE STUDENTS
    # -------------------------------------------------------------------------
    if len(students_df) > sample_size:
        sample_idx = np.random.choice(len(students_df), sample_size, replace=False)
        sample_df = students_df.iloc[sample_idx].copy()
    else:
        sample_df = students_df.copy()
    
    sample_df = sample_df.reset_index(drop=True)
    
    # Sort by score descending (highest first - they get processed first)
    sample_df = sample_df.sort_values(['Score'], ascending=False)
    sample_df = sample_df.reset_index(drop=True)
    
    n_students = len(sample_df)
    
    # -------------------------------------------------------------------------
    # SCHOOL CONFIGURATION
    # -------------------------------------------------------------------------
    school_order = [
        "Walter Payton", "Northside", "Whitney Young", "Jones", "Lane Tech",
        "Lindblom", "Hancock", "Brooks", "King", "Westinghouse", "South Shore"
    ]
    school_to_idx = {name: i for i, name in enumerate(school_order)}
    n_schools = len(school_order)
    
    # -------------------------------------------------------------------------
    # INITIALIZE SEAT COUNTS (actual seats / 10 for visualization)
    # -------------------------------------------------------------------------
    # Use actual school seats divided by 10 (matching 2000/22000 scale)
    seat_counts = {}
    for school_name in school_order:
        school = SCHOOLS.get(school_name)
        if school:
            total = max(school.seats // 10, 5)  # Actual seats / 10
            seat_counts[school_name] = {
                'Rank': max(int(total * 0.30), 1),
                1: max(int(total * 0.175), 1),
                2: max(int(total * 0.175), 1),
                3: max(int(total * 0.175), 1),
                4: max(int(total * 0.175), 1),
                'total': total
            }
    
    # -------------------------------------------------------------------------
    # FIGURE SETUP
    # -------------------------------------------------------------------------
    fig, ax = plt.subplots(figsize=(20, 12))

    # Layout parameters
    pool_x = 1.0
    school_start_x = 3.5
    school_spacing = 1.5
    unmatched_x = school_start_x + n_schools * school_spacing + 1

    y_min, y_max = 0, 10

    ax.set_xlim(-0.5, unmatched_x + 1.5)
    ax.set_ylim(y_min - 1.5, y_max + 2.5)
    ax.set_aspect('auto')
    ax.axis('off')

    # Title
    title_text = ax.text(
        (unmatched_x + 1) / 2, y_max + 2,
        'SEHS Serial Dictatorship Match',
        fontsize=14, fontweight='bold', ha='center'
    )

    # Phase indicator
    phase_text = ax.text(
        (unmatched_x + 1) / 2, y_max + 1.4,
        '',
        fontsize=11, ha='center', color='darkblue'
    )

    # -------------------------------------------------------------------------
    # COLUMN LABELS
    # -------------------------------------------------------------------------
    ax.text(pool_x, y_max + 0.7, 'APPLICANTS', fontsize=10, fontweight='bold',
            ha='center', va='bottom')
    ax.text(pool_x, y_max + 0.2, '(by score)', fontsize=8, ha='center',
            va='bottom', color='gray')

    # Region labels under applicant pool (matching the histogram columns)
    region_x_offsets = {'north': -0.6, 'loop': -0.2, 'west': 0.2, 'south': 0.6}
    region_labels = {'north': 'N', 'loop': 'L', 'west': 'W', 'south': 'S'}
    for region, offset in region_x_offsets.items():
        ax.text(pool_x + offset, y_min - 0.3, region_labels[region],
                fontsize=7, ha='center', va='top',
                color=REGION_BASE_COLORS[region], fontweight='bold')

    # School column indicators at top (simplified - names are at bottom)
    for i, school in enumerate(school_order):
        x = school_start_x + i * school_spacing
        ax.text(x, y_max + 0.5, '|', fontsize=12, ha='center', va='bottom',
                color='#999999')

    ax.text(unmatched_x, y_max + 0.7, 'UNMATCHED', fontsize=10, fontweight='bold',
            ha='center', va='bottom', color='#666666')

    # -------------------------------------------------------------------------
    # SEAT COUNT DISPLAYS AND CAP INDICATORS
    # -------------------------------------------------------------------------
    seat_texts = {}
    cap_indicators = {}  # Will hold cap indicator text objects

    for i, school in enumerate(school_order):
        x = school_start_x + i * school_spacing
        total = seat_counts[school]['total']
        short_name = SCHOOLS[school].short_name

        # School name (centered under column)
        ax.text(x, y_min - 0.2, short_name, fontsize=8, ha='center', va='top',
                fontweight='bold', color='#333333')

        # Seat count text (below school name)
        seat_texts[school] = ax.text(
            x, y_min - 0.55, f'[{total}]',
            fontsize=8, ha='center', va='top', color='darkblue'
        )

        # Cap indicator (below seat count) - shows which categories are capped
        cap_indicators[school] = ax.text(
            x, y_min - 0.85, '',
            fontsize=6, ha='center', va='top', color='red'
        )

    # -------------------------------------------------------------------------
    # SCORE AXIS LABELS (showing the exponential scale - bunched at bottom, spread at top)
    # -------------------------------------------------------------------------
    for score in [450, 550, 650, 750, 825, 875, 900]:
        y = score_to_y(score, y_min + 0.5, y_max - 0.5)
        ax.text(-0.3, y, str(score), fontsize=7, ha='right', va='center', color='gray')
        ax.axhline(y=y, xmin=0.01, xmax=0.05, color='lightgray', linewidth=0.5)

    # -------------------------------------------------------------------------
    # INITIALIZE STUDENT POSITIONS
    # Students start stacked by score (highest at top)
    # Regions are aligned in vertical columns to show distribution histogram
    # -------------------------------------------------------------------------

    # Region x-offsets for histogram-like display
    region_x_offsets = {
        'north': -0.6,   # Leftmost
        'loop':  -0.2,   # Center-left
        'west':   0.2,   # Center-right
        'south':  0.6,   # Rightmost
    }

    student_x = np.array([
        pool_x + region_x_offsets.get(row['Region'], 0)
        for _, row in sample_df.iterrows()
    ], dtype=float)

    student_y = np.array([
        score_to_y(row['Score'], y_min + 0.5, y_max - 0.5)
        for _, row in sample_df.iterrows()
    ])

    # Add small vertical jitter only (keep horizontal alignment for histogram effect)
    student_y += np.random.uniform(-0.05, 0.05, n_students)
    # Small horizontal jitter within region column
    student_x += np.random.uniform(-0.12, 0.12, n_students)

    # Initial colors (all students shown by region/tier)
    student_colors = [
        get_student_color(row['Region'], row['Tier'], 'tier')
        for _, row in sample_df.iterrows()
    ]

    # Track match type for color updates
    match_types = ['pool'] * n_students  # 'pool', 'rank', 'tier', 'unmatched'

    scatter = ax.scatter(student_x, student_y, c=student_colors, s=20,
                        edgecolors='white', linewidths=0.3, zorder=5)

    # -------------------------------------------------------------------------
    # TRACKING STATE
    # -------------------------------------------------------------------------
    remaining_seats = {
        school: {k: v for k, v in counts.items()}
        for school, counts in seat_counts.items()
    }

    dest_counts = defaultdict(int)  # Count per destination for stacking

    # Build processing order:
    # Phase 1: All students by score (for rank seats)
    # Phase 2: Students by tier, then by score within tier

    # Create tier-sorted indices for phase 2
    tier_indices = {t: [] for t in [1, 2, 3, 4]}
    for idx, row in sample_df.iterrows():
        tier_indices[row['Tier']].append(idx)

    # Processing queue: list of (student_idx, phase, tier_for_phase2)
    processing_queue = []

    # Phase 1: all students in score order (already sorted)
    for idx in range(n_students):
        processing_queue.append((idx, 'rank', None))

    # Phase 2: by tier, each tier in score order
    for tier in [1, 2, 3, 4]:
        for idx in tier_indices[tier]:
            processing_queue.append((idx, 'tier', tier))

    state = {
        'queue_pos': 0,
        'phase': 'rank',
        'current_tier': None,
        'matched': set(),  # Track which students are already matched
        'rank_phase_done': False,
    }

    # -------------------------------------------------------------------------
    # LEGEND
    # -------------------------------------------------------------------------
    legend_handles = [
        mpatches.Patch(color='#111111', label='Rank Match', alpha=0.9),
        mpatches.Patch(color=REGION_BASE_COLORS['north'], label='North', alpha=0.8),
        mpatches.Patch(color=REGION_BASE_COLORS['loop'], label='Loop', alpha=0.8),
        mpatches.Patch(color=REGION_BASE_COLORS['west'], label='West', alpha=0.8),
        mpatches.Patch(color=REGION_BASE_COLORS['south'], label='South', alpha=0.8),
    ]
    ax.legend(handles=legend_handles, loc='lower left', fontsize=5,
             title='Match Type / Region', framealpha=0.9)

    # Progress counter
    progress_text = ax.text(
        unmatched_x, y_min - 1.5,
        'Matched: 0 | Unmatched: 0',
        fontsize=9, ha='center', va='top'
    )

    matched_count = [0]
    unmatched_count = [0]

    # -------------------------------------------------------------------------
    # HELPER: Update cap indicators for a school
    # -------------------------------------------------------------------------
    def update_caps(school_name):
        caps = []
        if remaining_seats[school_name].get('Rank', 0) == 0:
            caps.append('R')
        for t in [1, 2, 3, 4]:
            if remaining_seats[school_name].get(t, 0) == 0:
                caps.append(f'T{t}')
        cap_indicators[school_name].set_text(' '.join(caps))

    # -------------------------------------------------------------------------
    # HELPER: Assign student to destination
    # -------------------------------------------------------------------------
    def assign_student(student_idx, dest_type, school_name=None, match_type='tier'):
        if dest_type == 'school' and school_name:
            dest_key = school_name
            x = school_start_x + school_to_idx[school_name] * school_spacing
        else:
            dest_key = 'unmatched'
            x = unmatched_x

        count = dest_counts[dest_key]
        dest_counts[dest_key] += 1

        # Stack from bottom, wrap if too many
        col_height = y_max - y_min - 1
        row = count % 40
        col_offset = (count // 40) * 0.25

        y = y_min + 0.3 + (row * 0.2)
        x_jitter = np.random.uniform(-0.25, 0.25) + col_offset

        student_x[student_idx] = x + x_jitter
        student_y[student_idx] = y
        match_types[student_idx] = match_type

        # Update color based on match type
        row_data = sample_df.iloc[student_idx]
        if match_type == 'rank':
            student_colors[student_idx] = (0.1, 0.1, 0.1, 0.9)  # Black for rank
        elif match_type == 'unmatched':
            student_colors[student_idx] = (0.5, 0.5, 0.5, 0.5)  # Gray for unmatched
        else:
            student_colors[student_idx] = get_student_color(
                row_data['Region'], row_data['Tier'], 'tier'
            )
    
    # -------------------------------------------------------------------------
    # ANIMATION UPDATE
    # -------------------------------------------------------------------------
    def update(frame):
        # Process multiple students per frame
        for _ in range(students_per_frame):
            if state['queue_pos'] >= len(processing_queue):
                # Animation complete
                title_text.set_text('SEHS Match Complete')
                phase_text.set_text(
                    f'Total Matched: {matched_count[0]} | Unmatched: {unmatched_count[0]}'
                )
                return scatter, title_text, phase_text, progress_text
            
            student_idx, phase, tier_target = processing_queue[state['queue_pos']]
            state['queue_pos'] += 1
            
            # Skip if already matched
            if student_idx in state['matched']:
                continue
            
            student = sample_df.iloc[student_idx]
            student_tier = student['Tier']
            prefs = student.get('Preferences', [])
            
            if phase == 'rank':
                # Phase 1: Rank-based matching
                if not state['rank_phase_done']:
                    title_text.set_text('CPS Selective Enrollment Match Process Simulation (Scaled down 10x)')
                    phase_text.set_text('Phase 1: RANK-BASED (Top 30% of seats)')
                    state['phase'] = 'rank'
                
                # Check if all rank seats are gone
                total_rank_seats = sum(
                    remaining_seats[s].get('Rank', 0) for s in school_order
                )
                if total_rank_seats == 0:
                    state['rank_phase_done'] = True
                    continue
                
                if isinstance(prefs, list) and len(prefs) > 0:
                    for school in prefs:
                        if school in remaining_seats:
                            if remaining_seats[school].get('Rank', 0) > 0:
                                # Match!
                                assign_student(student_idx, 'school', school, 'rank')
                                remaining_seats[school]['Rank'] -= 1
                                state['matched'].add(student_idx)
                                matched_count[0] += 1
                                
                                # Update displays
                                total_rem = sum(
                                    v for k, v in remaining_seats[school].items() 
                                    if k != 'total'
                                )
                                seat_texts[school].set_text(str(total_rem))
                                update_caps(school)
                                break
            
            else:
                # Phase 2: Tier-based matching
                state['phase'] = 'tier'
                state['current_tier'] = tier_target
                title_text.set_text('CPS Selective Enrollment Match Process Simulation (Scaled down 10x)')
                phase_text.set_text(f'Phase 2: TIER-BASED (Tier {tier_target})')
                
                # Only process if student is in current tier
                if student_tier != tier_target:
                    continue
                
                if isinstance(prefs, list) and len(prefs) > 0:
                    matched = False
                    for school in prefs:
                        if school in remaining_seats:
                            if remaining_seats[school].get(student_tier, 0) > 0:
                                # Match!
                                assign_student(student_idx, 'school', school, 'tier')
                                remaining_seats[school][student_tier] -= 1
                                state['matched'].add(student_idx)
                                matched_count[0] += 1
                                matched = True
                                
                                # Update displays
                                total_rem = sum(
                                    v for k, v in remaining_seats[school].items() 
                                    if k != 'total'
                                )
                                seat_texts[school].set_text(str(total_rem))
                                update_caps(school)
                                break
                    
                    if not matched:
                        # No seats available - unmatched
                        assign_student(student_idx, 'unmatched', match_type='unmatched')
                        state['matched'].add(student_idx)
                        unmatched_count[0] += 1
                else:
                    # No preferences - unmatched
                    assign_student(student_idx, 'unmatched', match_type='unmatched')
                    state['matched'].add(student_idx)
                    unmatched_count[0] += 1
        
        # Update scatter plot
        scatter.set_offsets(np.column_stack([student_x, student_y]))
        scatter.set_facecolors(student_colors)
        
        # Update progress
        progress_text.set_text(
            f'Matched: {matched_count[0]} | Unmatched: {unmatched_count[0]}'
        )
        
        return scatter, title_text, phase_text, progress_text
    
    # -------------------------------------------------------------------------
    # CREATE ANIMATION
    # -------------------------------------------------------------------------
    n_frames = (len(processing_queue) // students_per_frame) + 10
    
    anim = animation.FuncAnimation(
        fig, update, frames=n_frames,
        interval=interval_ms, blit=False, repeat=False
    )
    
    plt.close(fig)
    return anim


# ============================================================================
# RUN THE ANIMATION
# ============================================================================
print("Generating students for animation...")
# Generate 2000 students (scaled down from 22000 for visualization)
students_for_anim = generate_students(n=2000, seed=42)

print("Calculating preferences...")
students_for_anim['Preferences'] = students_for_anim.apply(calculate_preferences, axis=1)
students_for_anim['TieBreaker'] = np.random.random(len(students_for_anim))

print("Creating animation (this may take a moment)...")
print(f"Simulating {len(students_for_anim)} students competing for ~{sum(s.seats//10 for s in SCHOOLS.values())} seats")
print("The animation will show:")
print("  - Phase 1: Rank-based matching (30% of seats, black dots)")
print("  - Phase 2: Tier-based matching (Tiers 1-4, colored by region)")
print()

# Increase embed limit for larger animations
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 100  # 100 MB limit

anim = create_match_animation(
    students_for_anim,
    sample_size=2000,      # Use all 2000 generated students
    interval_ms=5,        # Very fast animation
    students_per_frame=10  # Process 20 students per frame for speed
)

print("Rendering animation... (this may take 30-60 seconds)")
HTML(anim.to_jshtml())


# Key Findings and Implications

## Summary of Insights

This analysis reveals several striking patterns in the CPS Selective Enrollment High School admissions process. The findings are organized into four major themes, each with significant implications for students, families, and policymakers.

---

## 1. The Tier System Creates Massive Score Advantages (40-113 Points)

The tier-based admissions policy generates substantial score differentials across socioeconomic groups:

| School | Tier 1 Cutoff | Tier 4 Cutoff | Tier Gap |
|--------|---------------|---------------|----------|
| **Lane Tech** | 746 | 859 | **113 pts** |
| **Northside** | 706.5 | 893 | **186.5 pts** |
| **Whitney Young** | 807 | 880 | **73 pts** |
| **Payton** | 796 | 898 | **102 pts** |
| **Jones** | 775 | 864 | **89 pts** |

**Interpretation:** A Tier 1 student can gain admission to Lane Tech with a score that would be 113 points below the Tier 4 threshold. This is the policy working as designed—creating pathways for students from under-resourced neighborhoods—but the magnitude is striking. At elite schools, tier assignment can be worth more than an entire letter grade on the composite score.

---

## 2. Published Averages Are Systematically Misleading (~140-150 Points)

The MLE analysis reveals a fundamental statistical illusion in CPS's published data:

| School | Published Average (Admitted) | MLE-Recovered Population Mean | Selection Bias |
|--------|------------------------------|-------------------------------|----------------|
| Payton T1 | 841.3 | ~719 | **~122 pts** |
| Lane Tech T1 | 758.2 | ~546 | **~212 pts** |
| Northside T4 | 894.3 | ~888 | **~6 pts** |

**The Truncation Problem:** CPS publishes the average score of *admitted* students, not all applicants. This is mathematically guaranteed to exceed the true population mean due to truncation at the cutoff. Our MLE recovers the hidden parameters by inverting the truncated normal distribution.

**Practical Impact:** A parent seeing "average: 876 at Payton" might conclude their 850-scoring child is below average. In reality, an 850 likely places them well above the applicant pool mean. The published statistics, while accurate, create a distorted picture of competitiveness.

---

## 3. Two Distinct SEHS Systems Exist Within One Policy

The data reveals a **bifurcated system** with fundamentally different competitive dynamics:

### Elite Schools (Payton, Northside, Young, Jones, Lane Tech)
- **T4 Hidden Mean:** $\mu \approx 830-870$
- **T4 Hidden SD:** $\sigma \approx 15-25$ (very tight)
- **Competition:** Extremely intense; 5-10 point differences are decisive
- **Geographic draw:** Citywide, especially North Side and Loop

### Regional Schools (Lindblom, Hancock, Brooks, King, Westinghouse, South Shore)
- **T4 Hidden Mean:** $\mu \approx 600-700$
- **T4 Hidden SD:** $\sigma \approx 60-100$ (much wider)
- **Competition:** Less intense; 30-50 point buffers are common
- **Geographic draw:** Primarily local neighborhoods

**Implication:** These are not simply "more selective" vs "less selective" versions of the same school type. They serve different populations with different score distributions, different geographic catchments, and different competitive pressures. Modeling them requires distinct parameter sets.

---

## 4. Three Regional Schools Exhibit Inverted Tier Gaps

A counterintuitive pattern emerges at Lindblom, King, and South Shore:

| School | Tier 1 Cutoff | Tier 4 Cutoff | Gap Direction |
|--------|---------------|---------------|---------------|
| Lindblom | 703 | 792 | Normal (+89) |
| King | 654 | 774 | Normal (+120) |
| South Shore | 576 | 711 | Normal (+135) |

*2024 data shows these gaps, but 2025 shows dramatic inversions at some schools.*

**Possible Explanations:**
1. **Geographic preference:** Tier 1 students in South/West Side neighborhoods may preferentially rank local schools higher, creating concentrated demand
2. **Elite school avoidance:** High-scoring Tier 1 students may not apply to Payton/Northside due to distance or cultural factors, redirecting competition to regional schools
3. **Information asymmetry:** Tier 4 families may be more likely to use regional schools as "safety" choices, diluting competition

This warrants further investigation—it suggests the tier system's effects are not uniform across the school landscape.

---

## Policy Implications

### For Parents and Students

1. **Don't be intimidated by published averages.** They are inflated by 100-200 points due to selection bias. Your score's percentile in the applicant pool is far higher than it appears.

2. **Tier 1/2 families have significant structural advantages at elite schools.** A Tier 1 student scoring 750 has realistic chances at Lane Tech; a Tier 4 student needs 860+.

3. **Regional schools offer genuine pathways for students scoring 550-750.** These are not "lesser" options—they are academically rigorous schools with less cutthroat admissions.

4. **Consider geographic distance seriously.** The simulation shows cross-region penalties in student preferences. A South Side student ranking only North Side schools may face implicit disadvantages.

### For CPS and Policymakers

1. **The tier system is achieving socioeconomic diversity**, but the magnitude of tier advantage varies wildly by school (40 pts at some, 180+ at others). Consider whether this variation is intentional.

2. **Publishing applicant-pool statistics** (not just admitted) would help families make informed choices and reduce the misleading nature of current data.

3. **The inverted gaps at regional schools** suggest uneven demand patterns worth investigating. Are Tier 1 students being funneled into a subset of schools?

4. **Model transparency:** The matching algorithm is strategy-proof, but the score distributions and preference models that determine outcomes are opaque. Greater transparency would build trust.

---

In [ ]:
# ============================================================================
# COMPREHENSIVE FINAL ANALYSIS: KEY VISUALIZATIONS
# ============================================================================
# This cell generates a series of publication-quality visualizations that
# highlight the most striking findings from the MLE and simulation analysis.
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns

# Use a clean style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# ----------------------------------------------------------------------------
# DATA: MLE-Recovered Parameters and Cutoffs
# ----------------------------------------------------------------------------

# MLE-recovered hidden population parameters (from individual school analyses)
mle_params = {
    'Walter Payton': {1: {'mu': 266.7, 'sigma': 200.0}, 4: {'mu': 770.0, 'sigma': 81.9}},
    'Northside': {1: {'mu': 448.0, 'sigma': 167.4}, 4: {'mu': 887.7, 'sigma': 3.4}},
    'Whitney Young': {1: {'mu': 548.4, 'sigma': 169.0}, 4: {'mu': 844.1, 'sigma': 23.5}},
    'Jones': {1: {'mu': 605.6, 'sigma': 110.7}, 4: {'mu': 838.8, 'sigma': 16.5}},
    'Lane Tech': {1: {'mu': 545.6, 'sigma': 109.7}, 4: {'mu': 830.6, 'sigma': 18.7}},
    'Lindblom': {1: {'mu': 490.4, 'sigma': 152.5}, 4: {'mu': 671.4, 'sigma': 86.5}},
    'Hancock': {1: {'mu': 642.6, 'sigma': 74.2}, 4: {'mu': 659.2, 'sigma': 81.6}},
    'Brooks': {1: {'mu': 430.0, 'sigma': 200.0}, 4: {'mu': 663.1, 'sigma': 73.8}},
    'King': {1: {'mu': 383.9, 'sigma': 200.0}, 4: {'mu': 690.3, 'sigma': 60.1}},
    'Westinghouse': {1: {'mu': 411.9, 'sigma': 173.0}, 4: {'mu': 603.1, 'sigma': 96.4}},
    'South Shore': {1: {'mu': 304.6, 'sigma': 196.9}, 4: {'mu': 590.1, 'sigma': 87.7}},
}

# Actual cutoffs (2024 calibration data)
cutoffs = {
    'Walter Payton': {1: 796, 4: 898, 'Rank': 900},
    'Northside': {1: 706.5, 4: 893, 'Rank': 895},
    'Whitney Young': {1: 807, 4: 880, 'Rank': 893},
    'Jones': {1: 775, 4: 864, 'Rank': 880},
    'Lane Tech': {1: 712, 4: 859, 'Rank': 877},
    'Lindblom': {1: 703, 4: 792, 'Rank': 827},
    'Hancock': {1: 746, 4: 773, 'Rank': 808},
    'Brooks': {1: 683, 4: 766, 'Rank': 825},
    'King': {1: 654, 4: 774, 'Rank': 808},
    'Westinghouse': {1: 653, 4: 737.5, 'Rank': 780},
    'South Shore': {1: 576, 4: 711, 'Rank': 777},
}

# Published averages (admitted students only)
published_avgs = {
    'Walter Payton': {1: 841.3, 4: 899.7},
    'Northside': {1: 768.5, 4: 894.3},
    'Whitney Young': {1: 846.0, 4: 887.4},
    'Jones': {1: 815.7, 4: 871.1},
    'Lane Tech': {1: 758.2, 4: 867.1},
    'Lindblom': {1: 720.1, 4: 667.4},
    'Hancock': {1: 779.3, 4: 807.9},
    'Brooks': {1: 729.0, 4: 747.7},
    'King': {1: 567.0, 4: 573.1},
    'Westinghouse': {1: 700.2, 4: 703.6},
    'South Shore': {1: 587.9, 4: 568.1},
}

elite_schools = ['Walter Payton', 'Northside', 'Whitney Young', 'Jones', 'Lane Tech']
regional_schools = ['Lindblom', 'Hancock', 'Brooks', 'King', 'Westinghouse', 'South Shore']
all_schools = elite_schools + regional_schools

# Short names for plotting
short_names = {
    'Walter Payton': 'Payton', 'Northside': 'Northside', 'Whitney Young': 'Young',
    'Jones': 'Jones', 'Lane Tech': 'Lane', 'Lindblom': 'Lindblom',
    'Hancock': 'Hancock', 'Brooks': 'Brooks', 'King': 'King',
    'Westinghouse': 'Westinghouse', 'South Shore': 'S. Shore'
}

# ============================================================================
# FIGURE 1: The Tier Gap - How Much Is Your Tier Worth?
# ============================================================================

fig1, ax1 = plt.subplots(figsize=(14, 8))

schools = all_schools
x = np.arange(len(schools))
width = 0.35

t1_cutoffs = [cutoffs[s][1] for s in schools]
t4_cutoffs = [cutoffs[s][4] for s in schools]
tier_gaps = [t4 - t1 for t1, t4 in zip(t1_cutoffs, t4_cutoffs)]

# Create grouped bar chart
bars1 = ax1.bar(x - width/2, t1_cutoffs, width, label='Tier 1 Cutoff', color='#2ecc71', alpha=0.8)
bars2 = ax1.bar(x + width/2, t4_cutoffs, width, label='Tier 4 Cutoff', color='#e74c3c', alpha=0.8)

# Add tier gap annotations
for i, (t1, t4, gap) in enumerate(zip(t1_cutoffs, t4_cutoffs, tier_gaps)):
    mid = (t1 + t4) / 2
    ax1.annotate(f'+{gap:.0f}', xy=(i, mid), ha='center', va='center',
                fontsize=9, fontweight='bold', color='#2c3e50',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor='gray', alpha=0.8))

# Highlight elite vs regional
ax1.axvline(x=4.5, color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
ax1.text(2, 920, 'ELITE SCHOOLS', ha='center', fontsize=11, fontweight='bold', color='#2c3e50')
ax1.text(7.5, 920, 'REGIONAL SCHOOLS', ha='center', fontsize=11, fontweight='bold', color='#2c3e50')

ax1.set_xlabel('School', fontsize=12)
ax1.set_ylabel('Cutoff Score', fontsize=12)
ax1.set_title('The Tier Advantage: Cutoff Score Gaps Between Tier 1 and Tier 4\n(Numbers show point advantage for Tier 1 students)', 
              fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels([short_names[s] for s in schools], rotation=45, ha='right')
ax1.set_ylim(500, 950)
ax1.legend(loc='lower right')
ax1.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('insight_1_tier_gaps.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure 1: Tier gaps range from 27 points (Hancock) to 186 points (Northside)")
print()

# ============================================================================
# FIGURE 2: Selection Bias - Published vs True Means
# ============================================================================

fig2, axes = plt.subplots(1, 2, figsize=(16, 7))

# Left panel: Tier 1
ax2a = axes[0]
schools_t1 = all_schools
x = np.arange(len(schools_t1))
width = 0.35

published_t1 = [published_avgs[s][1] for s in schools_t1]
true_t1 = [mle_params[s][1]['mu'] for s in schools_t1]
bias_t1 = [p - t for p, t in zip(published_t1, true_t1)]

bars1 = ax2a.bar(x - width/2, true_t1, width, label='True Population Mean (MLE)', color='#3498db', alpha=0.8)
bars2 = ax2a.bar(x + width/2, published_t1, width, label='Published Average (Admitted)', color='#e67e22', alpha=0.8)

# Add bias annotations for largest gaps
for i, bias in enumerate(bias_t1):
    if bias > 100:
        ax2a.annotate(f'+{bias:.0f}', xy=(i + width/2, published_t1[i] + 10), 
                     ha='center', fontsize=8, color='#c0392b', fontweight='bold')

ax2a.set_xlabel('School', fontsize=11)
ax2a.set_ylabel('Score', fontsize=11)
ax2a.set_title('Tier 1: Selection Bias in Published Averages', fontsize=12, fontweight='bold')
ax2a.set_xticks(x)
ax2a.set_xticklabels([short_names[s] for s in schools_t1], rotation=45, ha='right')
ax2a.set_ylim(200, 900)
ax2a.legend(loc='upper left', fontsize=9)
ax2a.grid(axis='y', alpha=0.3)

# Right panel: Tier 4
ax2b = axes[1]
published_t4 = [published_avgs[s][4] for s in schools_t1]
true_t4 = [mle_params[s][4]['mu'] for s in schools_t1]
bias_t4 = [p - t for p, t in zip(published_t4, true_t4)]

bars1 = ax2b.bar(x - width/2, true_t4, width, label='True Population Mean (MLE)', color='#3498db', alpha=0.8)
bars2 = ax2b.bar(x + width/2, published_t4, width, label='Published Average (Admitted)', color='#e67e22', alpha=0.8)

ax2b.set_xlabel('School', fontsize=11)
ax2b.set_ylabel('Score', fontsize=11)
ax2b.set_title('Tier 4: Selection Bias in Published Averages', fontsize=12, fontweight='bold')
ax2b.set_xticks(x)
ax2b.set_xticklabels([short_names[s] for s in schools_t1], rotation=45, ha='right')
ax2b.set_ylim(500, 950)
ax2b.legend(loc='upper left', fontsize=9)
ax2b.grid(axis='y', alpha=0.3)

plt.suptitle('The Selection Bias Illusion: What CPS Publishes vs. Reality\n(Published averages are truncated means that overstate typical applicant scores)', 
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('insight_2_selection_bias.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure 2: Selection bias inflates published T1 averages by 100-300+ points at elite schools")
print()

# ============================================================================
# FIGURE 3: Two SEHS Systems - Elite vs Regional Distribution Comparison
# ============================================================================

fig3, ax3 = plt.subplots(figsize=(14, 8))

from scipy.stats import norm

x_range = np.linspace(400, 920, 500)

# Plot T4 distributions for elite schools
elite_colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(elite_schools)))
for i, school in enumerate(elite_schools):
    mu = mle_params[school][4]['mu']
    sigma = mle_params[school][4]['sigma']
    y = norm.pdf(x_range, mu, sigma)
    ax3.fill_between(x_range, y, alpha=0.3, color=elite_colors[i])
    ax3.plot(x_range, y, linewidth=2, color=elite_colors[i], label=f'{short_names[school]} (μ={mu:.0f}, σ={sigma:.0f})')

# Plot T4 distributions for regional schools
regional_colors = plt.cm.Oranges(np.linspace(0.4, 0.9, len(regional_schools)))
for i, school in enumerate(regional_schools):
    mu = mle_params[school][4]['mu']
    sigma = mle_params[school][4]['sigma']
    y = norm.pdf(x_range, mu, sigma)
    ax3.fill_between(x_range, y, alpha=0.3, color=regional_colors[i])
    ax3.plot(x_range, y, linewidth=2, color=regional_colors[i], linestyle='--',
             label=f'{short_names[school]} (μ={mu:.0f}, σ={sigma:.0f})')

ax3.set_xlabel('Composite Score', fontsize=12)
ax3.set_ylabel('Probability Density', fontsize=12)
ax3.set_title('Two Distinct Systems: Tier 4 Score Distributions at Elite vs Regional Schools\n(MLE-Recovered Hidden Population Distributions)', 
              fontsize=13, fontweight='bold')
ax3.legend(loc='upper left', fontsize=8, ncol=2)
ax3.set_xlim(400, 920)
ax3.grid(alpha=0.3)

# Add annotation boxes
ax3.annotate('ELITE SCHOOLS\nTight distributions (σ ≈ 15-80)\nCentered around 830-890', 
            xy=(850, 0.08), fontsize=10, ha='center',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='#d5e8f7', edgecolor='#3498db', alpha=0.9))
ax3.annotate('REGIONAL SCHOOLS\nWide distributions (σ ≈ 60-100)\nCentered around 590-700', 
            xy=(650, 0.012), fontsize=10, ha='center',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='#fdebd0', edgecolor='#e67e22', alpha=0.9))

plt.tight_layout()
plt.savefig('insight_3_two_systems.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure 3: Elite T4 distributions are extremely tight; regional schools have 4-5x wider spreads")
print()

# ============================================================================
# FIGURE 4: The Sigma Story - Distribution Width by School Type
# ============================================================================

fig4, ax4 = plt.subplots(figsize=(12, 7))

# Collect sigma values
schools_ordered = elite_schools + regional_schools
t1_sigmas = [mle_params[s][1]['sigma'] for s in schools_ordered]
t4_sigmas = [mle_params[s][4]['sigma'] for s in schools_ordered]

x = np.arange(len(schools_ordered))
width = 0.35

colors_t1 = ['#27ae60' if s in elite_schools else '#f39c12' for s in schools_ordered]
colors_t4 = ['#2980b9' if s in elite_schools else '#e74c3c' for s in schools_ordered]

bars1 = ax4.bar(x - width/2, t1_sigmas, width, label='Tier 1 σ', color=colors_t1, alpha=0.8, edgecolor='white')
bars2 = ax4.bar(x + width/2, t4_sigmas, width, label='Tier 4 σ', color=colors_t4, alpha=0.8, edgecolor='white')

ax4.axvline(x=4.5, color='gray', linestyle='--', linewidth=2, alpha=0.7)
ax4.text(2, 210, 'ELITE', ha='center', fontsize=12, fontweight='bold')
ax4.text(7.5, 210, 'REGIONAL', ha='center', fontsize=12, fontweight='bold')

# Highlight the extremes
ax4.annotate('Northside T4:\nσ = 3.4 (!)', xy=(1 + width/2, 3.4), xytext=(1.5, 50),
            arrowprops=dict(arrowstyle='->', color='#c0392b'), fontsize=9, color='#c0392b')

ax4.set_xlabel('School', fontsize=12)
ax4.set_ylabel('Standard Deviation (σ)', fontsize=12)
ax4.set_title('Distribution Width (σ): How Spread Out Are Applicant Scores?\n(Lower σ = more competitive, tighter clustering at high scores)', 
              fontsize=13, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels([short_names[s] for s in schools_ordered], rotation=45, ha='right')
ax4.legend(loc='upper right')
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('insight_4_sigma_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure 4: Northside T4 has σ=3.4 (essentially everyone scores 885-895)")
print("         Regional schools have σ=60-100 (much more variance in applicant quality)")
print()

# ============================================================================
# FIGURE 5: Combined Summary - The Complete Picture
# ============================================================================

fig5, axes = plt.subplots(2, 2, figsize=(16, 14))

# Panel A: Tier Gap Heatmap
ax5a = axes[0, 0]
tier_gap_data = []
for school in all_schools:
    gap = cutoffs[school][4] - cutoffs[school][1]
    tier_gap_data.append({
        'School': short_names[school],
        'Type': 'Elite' if school in elite_schools else 'Regional',
        'Tier Gap': gap
    })
tier_gap_df = pd.DataFrame(tier_gap_data)

colors = ['#3498db' if t == 'Elite' else '#e74c3c' for t in tier_gap_df['Type']]
bars = ax5a.barh(tier_gap_df['School'], tier_gap_df['Tier Gap'], color=colors, alpha=0.8)
ax5a.set_xlabel('Tier Gap (T4 Cutoff - T1 Cutoff)', fontsize=11)
ax5a.set_title('A. Tier Advantage by School', fontsize=12, fontweight='bold')
ax5a.axvline(x=0, color='gray', linewidth=0.5)

for i, (gap, school) in enumerate(zip(tier_gap_df['Tier Gap'], tier_gap_df['School'])):
    ax5a.text(gap + 3, i, f'{gap:.0f}', va='center', fontsize=9)

# Panel B: Selection Bias (T1)
ax5b = axes[0, 1]
bias_data = []
for school in all_schools:
    pub = published_avgs[school][1]
    true = mle_params[school][1]['mu']
    bias_data.append({
        'School': short_names[school],
        'Bias': pub - true,
        'Type': 'Elite' if school in elite_schools else 'Regional'
    })
bias_df = pd.DataFrame(bias_data)

colors = ['#3498db' if t == 'Elite' else '#e74c3c' for t in bias_df['Type']]
bars = ax5b.barh(bias_df['School'], bias_df['Bias'], color=colors, alpha=0.8)
ax5b.set_xlabel('Selection Bias (Published - True Mean)', fontsize=11)
ax5b.set_title('B. How Much Published Averages Overstate Reality (Tier 1)', fontsize=12, fontweight='bold')
ax5b.axvline(x=100, color='orange', linestyle='--', alpha=0.7, label='100 pt bias')
ax5b.legend(loc='lower right', fontsize=9)

# Panel C: Mean comparison scatter
ax5c = axes[1, 0]
for school in all_schools:
    mu1 = mle_params[school][1]['mu']
    mu4 = mle_params[school][4]['mu']
    color = '#3498db' if school in elite_schools else '#e74c3c'
    marker = 'o' if school in elite_schools else 's'
    ax5c.scatter(mu1, mu4, s=150, c=color, marker=marker, alpha=0.8, edgecolors='white', linewidth=2)
    ax5c.annotate(short_names[school], (mu1, mu4), xytext=(5, 5), textcoords='offset points', fontsize=8)

ax5c.plot([200, 900], [200, 900], 'k--', alpha=0.3, label='Equal means line')
ax5c.set_xlabel('Tier 1 Population Mean (μ)', fontsize=11)
ax5c.set_ylabel('Tier 4 Population Mean (μ)', fontsize=11)
ax5c.set_title('C. Population Means: T1 vs T4 by School', fontsize=12, fontweight='bold')
ax5c.set_xlim(200, 900)
ax5c.set_ylim(500, 920)
ax5c.legend(loc='lower right')
ax5c.grid(alpha=0.3)

# Panel D: Sigma comparison scatter
ax5d = axes[1, 1]
for school in all_schools:
    sig1 = mle_params[school][1]['sigma']
    sig4 = mle_params[school][4]['sigma']
    color = '#3498db' if school in elite_schools else '#e74c3c'
    marker = 'o' if school in elite_schools else 's'
    ax5d.scatter(sig1, sig4, s=150, c=color, marker=marker, alpha=0.8, edgecolors='white', linewidth=2)
    ax5d.annotate(short_names[school], (sig1, sig4), xytext=(5, 5), textcoords='offset points', fontsize=8)

ax5d.set_xlabel('Tier 1 Population SD (σ)', fontsize=11)
ax5d.set_ylabel('Tier 4 Population SD (σ)', fontsize=11)
ax5d.set_title('D. Distribution Width: T1 vs T4 by School', fontsize=12, fontweight='bold')

# Add legend
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#3498db', markersize=12, label='Elite Schools'),
    Line2D([0], [0], marker='s', color='w', markerfacecolor='#e74c3c', markersize=12, label='Regional Schools')
]
ax5d.legend(handles=legend_elements, loc='upper right')
ax5d.grid(alpha=0.3)

plt.suptitle('Comprehensive Analysis: The Hidden Structure of SEHS Admissions', 
             fontsize=15, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('insight_5_comprehensive.png', dpi=150, bbox_inches='tight')
plt.show()

print("=" * 70)
print("ANALYSIS COMPLETE")
print("=" * 70)
print()
print("Key Takeaways:")
print("  1. Tier gaps range from 27 to 186 points - tier assignment is decisive")
print("  2. Published averages overstate applicant means by 100-300+ points")
print("  3. Elite and regional schools have fundamentally different distributions")
print("  4. Northside T4 is the most competitive (σ=3.4), South Shore the least (σ=87.7)")
print()
print("Figures saved:")
print("  - insight_1_tier_gaps.png")
print("  - insight_2_selection_bias.png")
print("  - insight_3_two_systems.png")
print("  - insight_4_sigma_comparison.png")
print("  - insight_5_comprehensive.png")
